<a href="https://colab.research.google.com/github/anaiy2004/ALI-Risk-Prediction/blob/main/MIMIC-EMR-ALI-Risk-Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget -r -N -c -np --user anaiysomalwar --ask-password https://physionet.org/files/mimiciii/1.4/    
#reading in files from MIMIC III (password protected)
#run this directly on terminal when using gcp
#output of code cell deleted for privacy

In [2]:
#allows for Pyspark / SQL use on Google Collab
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://apache.claz.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz
!tar xf spark-3.1.2-bin-hadoop3.2.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop3.2"
!pip3 install findspark
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [3]:
#https://github.com/dhlee4/AcuteOrganFailureInterventionModel/blob/master/mimic_hp_training_scale.py lines 117 - end of file -> first step of recreation calls mimic_run_experiment on line 131
#since mimic_run_experiment is called, we move to line 25 in that same file, where the class has the constructor
#on line 31, mimic_preprocesser is called is constructed, so we go to https://github.com/dhlee4/AcuteOrganFailureInterventionModel/blob/master/mimic_preprocess.py
#on line 8, mimic_data_abstracter is called so we go to https://github.com/dhlee4/AcuteOrganFailureInterventionModel/blob/master/mimic_data_abstracter.py -> does nothing but def methods
#on line 9 from mimic_preprocess, data_preprocessor is initialized https://github.com/dhlee4/AcuteOrganFailureInterventionModel/blob/master/data_preprocessor.py
#where data_preprocessor is initialized, preprocessor_gen is called

#Target Disease: 42731, 5849, 51881,5990
from pyspark.sql.functions import col,when,lit
# this target disease does not do anyything
target_disease=["51881"]


In [4]:
def add_demo():
        import pyspark
        from pyspark.sql.functions import datediff,col
        from pyspark.ml.feature import OneHotEncoder, StringIndexer
        from pyspark.ml.feature import VectorAssembler
        import pandas as pd

        cur_demo = pd.read_csv("physionet.org/files/mimiciii/1.4/PATIENTS.csv.gz", low_memory= False)
        patientsFinalColumns = ["SUBJECT_ID", "DOB", "GENDER"]
        cur_demo = cur_demo[cur_demo.columns.intersection(patientsFinalColumns)]
        
        cur_pts = pd.read_csv("physionet.org/files/mimiciii/1.4/ADMISSIONS.csv.gz",  low_memory= False)
        admissionsFinalColumnNames = ["SUBJECT_ID", "HADM_ID", "ADMITTIME", "ADMISSION_TYPE", "ADMISSION_LOCATION", "INSURANCE", "LANGUAGE", "RELIGION", "MARITAL_STATUS", "ETHNICITY"]
        cur_pts = cur_pts[cur_pts.columns.intersection(admissionsFinalColumnNames)]

        from pyspark.context import SparkContext
        from pyspark.sql.session import SparkSession
        sc = SparkContext.getOrCreate();
        spark = SparkSession(sc)
        cur_pts.LANGUAGE = cur_pts.LANGUAGE.astype(str)
        cur_pts.MARITAL_STATUS = cur_pts.MARITAL_STATUS.astype(str)
        cur_pts.RELIGION = cur_pts.RELIGION.astype(str)
        cur_demo = spark.createDataFrame(cur_demo)
        cur_pts = spark.createDataFrame(cur_pts)
        merged_demo = cur_demo.join(cur_pts,"SUBJECT_ID").drop("SUBJECT_ID")
        merged_demo = merged_demo.withColumn("AGE",datediff("ADMITTIME","DOB")/365.0).withColumn("AGE",when(col("AGE")>90,90).otherwise(col("AGE"))).drop("ADMITTIME","DOB").where("AGE > 18").fillna("N/A")

        target_col = merged_demo.columns
        target_col.remove("AGE")
        target_col.remove("HADM_ID")
        target_col.sort()
        vector_target = ["AGE"]
        demo_col_list = ["AGE"]
        for cat_col in target_col:
            SI_model= StringIndexer(inputCol=cat_col, outputCol="SI_{0}".format(cat_col)).fit(merged_demo)
            demo_col_list = demo_col_list+[demo_var+"||"+demo_info for demo_var, demo_info in (zip([cat_col]*len(SI_model.labels),SI_model.labels))]
            merged_demo = SI_model.transform(merged_demo)
            merged_demo = OneHotEncoder(inputCol="SI_{0}".format(cat_col),outputCol="OH_{0}".format(cat_col), dropLast=False).fit(merged_demo).transform(merged_demo)
            vector_target.append("OH_{0}".format(cat_col))

        import json

        return_df = VectorAssembler(inputCols=vector_target,outputCol="demo_feature").transform(merged_demo)
        return return_df.withColumnRenamed("HADM_ID", "ID").select("ID","demo_feature")

In [5]:
#x = add_demo()

In [6]:
#look at https://github.com/dhlee4/AcuteOrganFailureInterventionModel/blob/master/mimic_data_abstracter.py
####################### -> Part 2 of the Code -> equivalent to get_df_df method
#https://github.com/dhlee4/AcuteOrganFailureInterventionModel/blob/master/mimic_data_abstracter.py lines 22 -> 39
def get_def_df():
  import pandas as pd
  cur_chart_df = pd.read_csv("physionet.org/files/mimiciii/1.4/D_ITEMS.csv.gz")
  cur_chart_df = cur_chart_df[cur_chart_df["LINKSTO"] == "chartevents"]
  cur_chart_dfFinalColumns = ["ITEMID","LABEL"]
  cur_chart_df = cur_chart_df[cur_chart_df.columns.intersection(cur_chart_dfFinalColumns)]
  cur_chart_df_AddedList = []
  for i in range(len(cur_chart_df)):
    cur_chart_df_AddedList.append( "VITAL") #this is the .alias lit() segment in SQL
  cur_chart_df["SOURCE"] = cur_chart_df_AddedList
  cur_chart_df.head()

  cur_med_df = pd.read_csv("physionet.org/files/mimiciii/1.4/D_ITEMS.csv.gz")
  cur_med_df = cur_med_df[cur_med_df["LINKSTO"] == "inputevents_mv"]
  cur_med_dfFinalColumns = ["ITEMID", "LABEL"]
  cur_med_df = cur_med_df[cur_med_df.columns.intersection(cur_med_dfFinalColumns)]
  cur_med_df_AddedList = []
  for i in range(len(cur_med_df)):
    cur_med_df_AddedList.append("MED") #this is the .alias lit() segment in SQL
  cur_med_df["SOURCE"] = cur_med_df_AddedList
  cur_med_df.head()

  cur_proc_df = pd.read_csv("physionet.org/files/mimiciii/1.4/D_ITEMS.csv.gz")
  cur_proc_df = cur_proc_df[cur_proc_df["LINKSTO"] == "procedureevents_mv"]
  cur_proc_dfFinalColumns = ["ITEMID", "LABEL"]
  cur_proc_df = cur_proc_df[cur_proc_df.columns.intersection(cur_proc_dfFinalColumns)]
  cur_proc_df_AddedList = []
  for i in range(len(cur_proc_df)):
    cur_proc_df_AddedList.append("PROC") #this is the .alias lit() segment in SQL
  cur_proc_df["SOURCE"] = cur_proc_df_AddedList
  cur_proc_df.head()

  cur_lab_df = pd.read_csv("physionet.org/files/mimiciii/1.4/D_ITEMS.csv.gz")
  cur_lab_dfFinalColumns = ["ITEMID", "LABEL"]
  cur_lab_df = cur_lab_df[cur_lab_df.columns.intersection(cur_lab_dfFinalColumns)]
  cur_lab_df_AddedList = []
  for i in range(len(cur_lab_df)):
    cur_lab_df_AddedList.append("LAB") #this is the .alias lit() segment in SQL
  cur_lab_df["SOURCE"] = cur_lab_df_AddedList
  cur_lab_df.head()

  #double check whether \ in SQL is just a line break
  import pandas as pd
  #what does lit($$) mean? I am just adding together things instead of the .concat method, but there is lit($$) between them, which I am assuming is " "
  cur_cpt_df = pd.read_csv("physionet.org/files/mimiciii/1.4/CPTEVENTS.csv.gz").fillna("")
  cur_cpt_df_renamedCPT_CD = cur_cpt_df["CPT_CD"]
  cur_cpt_df["ITEMID"] = cur_cpt_df_renamedCPT_CD
  cur_cpt_df_concatdescription = []
  for i in range(len(cur_cpt_df)):
    cur_cpt_df_concatdescription.append(cur_cpt_df["SECTIONHEADER"][i] + " " + cur_cpt_df["SUBSECTIONHEADER"][i] + cur_cpt_df["DESCRIPTION"][i]) #double check this
  cur_cpt_df["LABEL"] = cur_cpt_df_concatdescription
  cur_cpt_df_AddedList = []
  for i in range(len(cur_cpt_df)):
    cur_cpt_df_AddedList.append("CPT") #this is the .alias lit() segment in SQL
  cur_cpt_df["SOURCE"] = cur_cpt_df_AddedList
  cur_cpt_dfFinalColumns = ["ITEMID", "LABEL", "SOURCE"]
  cur_cpt_df = cur_cpt_df[cur_cpt_df.columns.intersection(cur_cpt_dfFinalColumns)]
  cur_cpt_df = cur_cpt_df.drop_duplicates(inplace = False)
  cur_cpt_df.head()
  # currently on line 32 of github

  cur_icd_df = pd.read_csv("physionet.org/files/mimiciii/1.4/D_ICD_DIAGNOSES.csv.gz")
  cur_icd_df_renamedICD9Code = cur_icd_df["ICD9_CODE"]
  cur_icd_df["ITEMID"] = cur_icd_df_renamedICD9Code
  cur_icd_df_renamedLongTitle = cur_icd_df["LONG_TITLE"]
  cur_icd_df["LABEL"] = cur_icd_df_renamedLongTitle
  cur_icd_df_AddedList = []
  for i in range(len(cur_icd_df)):
    cur_icd_df_AddedList.append("ICD_DIAGNOSIS") #this is the .alias lit() segment in SQL
  cur_icd_df["SOURCE"] = cur_icd_df_AddedList
  cur_icd_dfFinalColumns = ["ITEMID", "LABEL", "SOURCE"]
  cur_icd_df = cur_icd_df[cur_icd_df.columns.intersection(cur_icd_dfFinalColumns)]
  cur_icd_df.head()

  #what are we merging the dataframes by? -> assuming that we just stack them and remove duplicates, which is what the union operator seems to be
  def_df = pd.concat([cur_med_df, cur_proc_df, cur_chart_df, cur_lab_df, cur_cpt_df,cur_icd_df]).drop_duplicates()
  print(len(def_df))
  def_df.head()
  return def_df

In [7]:
#y = get_def_df()

In [8]:
import pandas as pd

In [9]:
####################### -> Part 3 of the Code -> equivalent to the remove_dnr_pts and get_obs_df method
#https://github.com/dhlee4/AcuteOrganFailureInterventionModel/blob/master/mimic_data_abstracter.py lines 45 -> 73
#implement get_obs_df on line 75 to create cur_chart for line 45
#cur_dnr_inst = cur_chart_df[cur_chart_df["ITEMID"] == 223758]
#cur_dnr_inst = cur_dnr_inst[cur_dnr_inst["VALUE"].isin(cur_dnr_assertion)]
#beginning implementation of get_obs_df method -> lines 75 - 111

import pandas as pd
def remove_dnr_pts(cur_chart = pd.DataFrame()):
  # double check whether get_obs_df is called first and that this if statement is just as a safety measure, because if remove_dnr_pts is called first
  #then, it would call get_obs_df which would call remove_dnr_points w/o the if statement, store it in cur_chart, and go through the non-if statement again
  if(cur_chart.empty):
    cur_chart = get_obs_df()

  cur_dnr_assertion = ["DNR (do not resuscitate)", "DNR / DNI", "Comfort measures only", "DNI (do not intubate)"]
  cur_dnr_inst = cur_chart[cur_chart["ITEMID"] == 223758]
  cur_dnr_inst = cur_dnr_inst[cur_dnr_inst["VALUE"].isin(cur_dnr_assertion)]
  cur_dnr_inst = cur_dnr_inst.groupby("ID").TIME_OBS.agg(['min'])
  cur_dnr_inst = cur_dnr_inst.rename(columns = {"min" : "DNR_TIME"})
  cur_chart = cur_chart.merge(cur_dnr_inst, on = "ID", how = "left")
  print(cur_dnr_inst.shape)
  del cur_dnr_inst
  cur_chart = cur_chart.head(int(len(cur_chart)/15)) # MAKE SURE TO COMMENT OUT CHANGED FROM 200!!!!!!!!!!!!!!!!!!!!!!!!
  #double check correct lambda function
  cur_chart = cur_chart[cur_chart['DNR_TIME'].apply(lambda x: str(x) == "NaT" or cur_chart["DNR_TIME"] > cur_chart["TIME_OBS"])]
  print(cur_chart.head())
  cur_chart = cur_chart.drop(columns = ["DNR_TIME"])
  pts_icu_stay = get_icu_stay() #continue this method, check whether to uncomment final code from get_obs_df
  cur_chart = cur_chart.merge(pts_icu_stay, on = "ID")
  cur_chart = cur_chart[cur_chart["INTIME"] <= cur_chart["TIME_OBS"]]
  cur_chart = cur_chart[cur_chart["OUTTIME"] >= cur_chart["TIME_OBS"]]
  cur_chart = cur_chart.drop(columns = ["INTIME", "OUTTIME"])
  return cur_chart

def get_obs_df():
  import pandas as pd
  cur_vital = pd.read_csv("physionet.org/files/mimiciii/1.4/CHARTEVENTS.csv.gz", usecols = ["HADM_ID", "VALUE", "CHARTTIME", "ITEMID"], dtype = {"HADM_ID" : "int32", "ITEMID" : "int32"})
  cur_vital = cur_vital.rename(columns = {"HADM_ID" : "ID", "CHARTTIME" : "TIME_OBS"})
  cur_vital_AddedList = []
  for i in range(len(cur_vital)):
    cur_vital_AddedList.append("VITAL") #this is the .alias lit() segment in SQL
  cur_vital["SOURCE"] = cur_vital_AddedList
  print(cur_vital.info())
  print(cur_vital.memory_usage())
  cur_vital.head()

  import pandas as pd
  cur_lab = pd.read_csv("physionet.org/files/mimiciii/1.4/LABEVENTS.csv.gz", usecols = ["HADM_ID", "VALUE", "CHARTTIME", "ITEMID"])
  cur_lab = cur_lab.rename(columns = {"HADM_ID" : "ID", "CHARTTIME" : "TIME_OBS"})
  cur_lab_AddedList = []
  for i in range(len(cur_lab)):
    cur_lab_AddedList.append("LAB") #this is the .alias lit() segment in SQL
  cur_lab["SOURCE"] = cur_lab_AddedList
  print(cur_lab.info())
  print(cur_lab.memory_usage())
  cur_lab.head()
  cur_vital = cur_vital.head(int(len(cur_vital)/40)) # MAKE SURE TO COMMENT OUT  -> CHANGED FROM 40

  merged_obs = pd.concat([cur_vital, cur_lab])
  merged_obs.head()

  #continuing get_obs_df method -> line 103 to 105
  # TEST THIS PART OF THE CODE
  merged_obs = pd.merge(merged_obs, get_icu_stay(), on='ID')
  merged_obs["INTIME"] = pd.to_datetime(merged_obs["INTIME"])
  merged_obs["OUTTIME"] = pd.to_datetime(merged_obs["OUTTIME"])
  merged_obs["TIME_OBS"] = pd.to_datetime(merged_obs["TIME_OBS"])
  merged_obs = merged_obs[merged_obs["INTIME"] < merged_obs["OUTTIME"]]
  merged_obs = merged_obs[merged_obs["TIME_OBS"] <= merged_obs["OUTTIME"]].drop(columns = ["INTIME", "OUTTIME"])
  merged_obs.head()
  # DOUBLE CHECK WHETHER TO INCLUDE THIS, maybe comment out for testing
  print("check!!!")
  merged_obs = remove_dnr_pts(cur_chart = merged_obs)

  return merged_obs

In [10]:
#implementing the get_icu_stay method, which is a helper method for get_obs_df (line 103 /216 -222)
def get_icu_stay():
  import pandas as pd
  get_icu_stay_df = pd.read_csv("physionet.org/files/mimiciii/1.4/ICUSTAYS.csv.gz", usecols = ["HADM_ID", "INTIME", "OUTTIME", "DBSOURCE"])
  get_icu_stay_df = get_icu_stay_df[get_icu_stay_df["DBSOURCE"] == "metavision"]
  get_icu_stay_df = get_icu_stay_df.rename(columns = {"HADM_ID" : "ID"})
  get_icu_stay_df["INTIME"] = pd.to_datetime(get_icu_stay_df["INTIME"])
  get_icu_stay_df["OUTTIME"] = pd.to_datetime(get_icu_stay_df["OUTTIME"])
  get_icu_stay_df.head()
  return get_icu_stay_df

In [11]:
#obsDf = get_obs_df() use only top comment
#cur_chart = remove_dnr_pts(obsDf)

In [12]:
#obsDf.head()
#check why dbsource x and y exist and are not merged -> should be baesd on the icu_stay new code for the method

In [13]:
def get_action_df():
  cur_med = pd.read_csv("physionet.org/files/mimiciii/1.4/INPUTEVENTS_MV.csv.gz", usecols = ["HADM_ID", "STARTTIME", "ITEMID"])
  cur_med_AddedList = []
  for i in range(len(cur_med)):
    cur_med_AddedList.append("MED") #this is the .alias lit() segment in SQL
  cur_med["SOURCE"] = cur_med_AddedList
  cur_med = cur_med.rename(columns = {"HADM_ID" : "ID", "STARTTIME" : "TIME_OBS"})

  cur_cpt = pd.read_csv("physionet.org/files/mimiciii/1.4/CPTEVENTS.csv.gz", usecols = ["HADM_ID", "CHARTDATE", "CPT_NUMBER"])
  cur_cpt_AddedList = []
  for i in range(len(cur_cpt)):
    cur_cpt_AddedList.append("CPT") #this is the .alias lit() segment in SQL
  cur_cpt["SOURCE"] = cur_cpt_AddedList
  cur_cpt = cur_cpt.rename(columns = {"HADM_ID" : "ID", "CHARTDATE" : "TIME_OBS", "CPT_NUMBER" : "ITEMID"})

  cur_proc = pd.read_csv("physionet.org/files/mimiciii/1.4/PROCEDUREEVENTS_MV.csv.gz", usecols = ["HADM_ID", "STARTTIME", "ITEMID"])
  cur_proc_AddedList = []
  for i in range(len(cur_proc)):
    cur_proc_AddedList.append("PROC") #this is the .alias lit() segment in SQL
  cur_proc["SOURCE"] = cur_proc_AddedList
  cur_proc = cur_proc.rename(columns = {"HADM_ID" : "ID", "STARTTIME" : "TIME_OBS"})

  ret_df = None
  for cur_df in [cur_med,cur_cpt,cur_proc]:
    #double check this if statement to ensure same purpose
    if(not cur_df.empty):
      if ret_df is not None:
        ret_df = pd.concat([ret_df, cur_df])
      else:
        ret_df = cur_df
  return ret_df.drop_duplicates()
      


In [14]:
#x = get_action_df()
#x.head()

In [15]:
def get_terminal_df():
  cur_df = pd.read_csv("physionet.org/files/mimiciii/1.4/DIAGNOSES_ICD.csv.gz", usecols = ["HADM_ID", "ICD9_CODE"])
  ret_df = cur_df.rename(columns = {"HADM_ID" : "ID"})
  return ret_df

In [16]:
#y = get_terminal_df()
#y.head()

In [17]:
def get_hospital_death():  
  ret_df = pd.read_csv("physionet.org/files/mimiciii/1.4/ADMISSIONS.csv.gz", usecols = ["HADM_ID", "HOSPITAL_EXPIRE_FLAG"])
  ret_df = ret_df.rename(columns = {"HADM_ID" : "ID", "HOSPITAL_EXPIRE_FLAG" : "IS_DEAD"})
  return ret_df

In [18]:
#z = get_hospital_death()
#z.head()

In [19]:
def get_action_itemids():
  cur_target_action_cpt_rdd = spark.read.option("header",True) \
     .csv("/content/physionet.org/files/mimiciii/1.4/CPTEVENTS.csv.gz") \
            .where(col("CHARTDATE").isNotNull()).where("SECTIONHEADER <> 'Evaluation and management'")\
            .select("CPT_CD").rdd.flatMap(list).map(lambda x: {"SOURCE":"CPT","ITEMID":x})

  cur_target_action_med_rdd = spark.read.option("header",True) \
     .csv("/content/physionet.org/files/mimiciii/1.4/D_ITEMS.csv.gz")\
            .where("DBSOURCE == 'metavision' and LINKSTO = 'inputevents_mv'")\
            .where(col("CATEGORY").isin(["Dialysis", "2-Ventilation", "Blood Products/Colloids", "4-Procedures"
                                                        , "1-Intubation/Extubation","3-Significant Events", "Medications"]))\
            .select("ITEMID").rdd.flatMap(list).map(lambda x:{"SOURCE":"MED","ITEMID":x})

  cur_target_action_proc_rdd = spark.read.option("header",True) \
     .csv("/content/physionet.org/files/mimiciii/1.4/D_ITEMS.csv.gz")\
            .where("DBSOURCE == 'metavision' and LINKSTO = 'procedureevents_mv'")\
            .where(col("CATEGORY").isin(["Dialysis", "2-Ventilation", "Blood Products/Colloids", "4-Procedures"
                                                        , "1-Intubation/Extubation","3-Significant Events", "Medications"]))\
            .select("ITEMID").rdd.flatMap(list).map(lambda x:{"SOURCE":"PROC","ITEMID":x})

  ret_df = spark.sparkContext.union([cur_target_action_cpt_rdd,cur_target_action_med_rdd,cur_target_action_proc_rdd]).\
            toDF().distinct()
  return ret_df

In [20]:
#x = get_action_itemids()

In [21]:
#pandasDF = x.toPandas()
#print(pandasDF)

In [22]:
#DBG methods were coppied and pasted; irrelevant for this purpose    
def get_target_ids_list_DBG(self):
  return self.cur_target_id

def get_obs_df_DBG(self,target_id):
  if type(self) == data_abstracter:
      raise NotImplementedError("Method need to be called in sub-class but currently called in base class")

  '''
  :param target_id: target ID used for debug run
  :return:
  '''
  import pyspark
  #maybe this can be moved to main method?
  try:
      return self.spark.read.parquet(self.cached_obs_df+self.dbg_post_fix)
  except pyspark.sql.utils.AnalysisException as ex:
      template = "An exception of type {0} occurred. Arguments:\n{1!r}"
      message = template.format(type(ex).__name__, ex.args)
      self.logger.info(message)
      self.logger.info("PROCESS")
  from pyspark.sql.functions import col
  cur_df = self.get_obs_df()
  cur_df.where(col("ID").isin(target_id)).write.save(self.cached_obs_df+self.dbg_post_fix)
  return self.spark.read.parquet(self.cached_obs_df+self.dbg_post_fix)

def get_action_df_DBG(self,target_id):
  if type(self) == data_abstracter:
      raise NotImplementedError("Method need to be called in sub-class but currently called in base class")

  '''
  :param target_id:target ID used for debug run
  :return:
  '''
  import pyspark

  try:
      return self.spark.read.parquet(self.cached_action_df+self.dbg_post_fix)
  except pyspark.sql.utils.AnalysisException as ex:
      template = "An exception of type {0} occurred. Arguments:\n{1!r}"
      message = template.format(type(ex).__name__, ex.args)
      self.logger.info(message)
      self.logger.info("PROCESS")
  from pyspark.sql.functions import col
  cur_df = self.get_action_df()
  cur_df.where(col("ID").isin(target_id)).write.save(self.cached_action_df+self.dbg_post_fix)
  return self.spark.read.parquet(self.cached_action_df+self.dbg_post_fix)

def get_terminal_df_DBG(self,target_id):
  if type(self) == data_abstracter:
      raise NotImplementedError("Method need to be called in sub-class but currently called in base class")

  '''
  :param target_id:target ID used for debug run
  :return:
  '''
  import pyspark
  try:
      return self.spark.read.parquet(self.cached_terminal_df+self.dbg_post_fix)
  except pyspark.sql.utils.AnalysisException as ex:
      template = "An exception of type {0} occurred. Arguments:\n{1!r}"
      message = template.format(type(ex).__name__, ex.args)
      self.logger.info(message)
      self.logger.info("PROCESS")
  from pyspark.sql.functions import col
  cur_df = self.get_terminal_df()
  cur_df.where(col("ID").isin(target_id)).write.save(self.cached_terminal_df+self.dbg_post_fix)
  return self.spark.read.parquet(self.cached_terminal_df+self.dbg_post_fix)

def get_target_ids_list():
  from pyspark.context import SparkContext
  from pyspark.sql.session import SparkSession
  sc = SparkContext.getOrCreate();
  spark = SparkSession(sc)
  obs_df = get_obs_df()
  obs_df.VALUE = obs_df.VALUE.astype(str)
  cur_target_id_list = spark.createDataFrame(obs_df)
  cur_target_id_list = cur_target_id_list.select("ID").distinct().rdd.flatMap(list).collect()
  return cur_target_id_list

In [23]:
def get_target_test_id(tr_te_prop = 0.1):

        import json
        cur_list = get_target_ids_list()
        from random import shuffle
        shuffle(cur_list)
        target_test_id_list = cur_list[:int(len(cur_list)*tr_te_prop)]
        return target_test_id_list

def get_target_tr_val_id(tr_val_prop = 0.2):
        import json
        cur_all_ids = get_target_ids_list()
        cur_test_list = get_target_test_id()
        from random import shuffle
        cur_tr_ids = list(set(cur_all_ids).difference(set(cur_test_list)))
        shuffle(cur_tr_ids)
        target_val_ids = cur_tr_ids[:int(len(cur_tr_ids)*tr_val_prop)]
        target_train_ids = cur_tr_ids[int(len(cur_tr_ids)*tr_val_prop):]
        return {"TR":target_train_ids, "VAL":target_val_ids}


In [24]:
def mimic_preprocess_init():
  #data_preprocesser_init() -> does nothing
  return mimic_data_abstracter_init()

In [25]:
def data_abstracter_init():
  import pyspark
  from pyspark.sql.session import SparkSession
  from pyspark import SparkConf
  def_df = get_def_df()
  obs_df = get_obs_df()

  from pyspark.sql.functions import col
  cur_target_id = get_target_ids_list()
  obs_df = obs_df[obs_df["ID"].isin(cur_target_id)]
  action_df = get_action_df()
  action_df = action_df[action_df["ID"].isin(cur_target_id)]
  terminal_df = get_terminal_df()
  terminal_df = terminal_df[terminal_df["ID"].isin(cur_target_id)]

  return cur_target_id, obs_df, action_df, terminal_df, def_df

In [26]:
def mimic_data_abstracter_init():
  return data_abstracter_init()

In [27]:
#main run
from pyspark import SparkConf
from pyspark.sql.session import SparkSession
import pyspark
cur_target_id, obs_df, action_df, terminal_df, def_df = mimic_preprocess_init()
#data_run_experiment_init() -> does nothing

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DtypeWarning: Columns (4,5,7,11) have mixed types.Specify dtype option on import or set low_memory=False.
  This is separate from the ipykernel package so we can avoid doing imports until


37878


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  This is separate from the ipykernel package so we can avoid doing imports until


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 330712483 entries, 0 to 330712482
Data columns (total 5 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   ID        int32 
 1   ITEMID    int32 
 2   TIME_OBS  object
 3   VALUE     object
 4   SOURCE    object
dtypes: int32(2), object(3)
memory usage: 9.9+ GB
None
Index              128
ID          1322849932
ITEMID      1322849932
TIME_OBS    2645699864
VALUE       2645699864
SOURCE      2645699864
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27854055 entries, 0 to 27854054
Data columns (total 5 columns):
 #   Column    Dtype  
---  ------    -----  
 0   ID        float64
 1   ITEMID    int64  
 2   TIME_OBS  object 
 3   VALUE     object 
 4   SOURCE    object 
dtypes: float64(1), int64(1), object(3)
memory usage: 1.0+ GB
None
Index             128
ID          222832440
ITEMID      222832440
TIME_OBS    222832440
VALUE       222832440
SOURCE      222832440
dtype: int64
check!!!
(0, 1)
         ID  ITEMID  

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 330712483 entries, 0 to 330712482
Data columns (total 5 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   ID        int32 
 1   ITEMID    int32 
 2   TIME_OBS  object
 3   VALUE     object
 4   SOURCE    object
dtypes: int32(2), object(3)
memory usage: 9.9+ GB
None
Index              128
ID          1322849932
ITEMID      1322849932
TIME_OBS    2645699864
VALUE       2645699864
SOURCE      2645699864
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27854055 entries, 0 to 27854054
Data columns (total 5 columns):
 #   Column    Dtype  
---  ------    -----  
 0   ID        float64
 1   ITEMID    int64  
 2   TIME_OBS  object 
 3   VALUE     object 
 4   SOURCE    object 
dtypes: float64(1), int64(1), object(3)
memory usage: 1.0+ GB
None
Index             128
ID          222832440
ITEMID      222832440
TIME_OBS    222832440
VALUE       222832440
SOURCE      222832440
dtype: int64
check!!!
(0, 1)
         ID  ITEMID  

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  This is separate from the ipykernel package so we can avoid doing imports until


In [28]:
def run_preprocessor(obs_df = obs_df):
  from pyspark.context import SparkContext
  from pyspark.sql.session import SparkSession
  from pyspark.sql.functions import col,split,struct, date_add
  sc = SparkContext.getOrCreate();
  spark = SparkSession(sc)
  obs_df.VALUE = obs_df.VALUE.astype(str)
  obs_df = spark.createDataFrame(obs_df)
  cur_obs_bf_dropna = obs_df.select("ID", "ITEMID", "VALUE", "TIME_OBS",
                                          split("TIME_OBS", "\ ").getItem(0).alias("DATE_OBS")) \
            .withColumn("TIME_SPAN", struct(col("DATE_OBS").cast("timestamp").alias("TIME_FROM") \
                                            , date_add("DATE_OBS", 1).cast("timestamp").alias("TIME_TO")))


  cur_obs = cur_obs_bf_dropna.dropna()
  return run_remaining(cur_obs)

In [29]:
def run_remaining(cur_obs):
  from pyspark.sql.functions import count,col
  cur_obs.groupBy("ITEMID").agg(count("*").alias("cnt")).orderBy(col("cnt").desc()).show(300)
  num_cat_tagged = num_cat_tagger(cur_obs)
  cat_raw_filtered, cat_voca_list = cat_frequency_filter(num_cat_tagged.where("IS_CAT == 1"))
  num_raw_filtered, num_ref_list = num_iqr_filter(num_cat_tagged.where("IS_CAT == 0"))
  num_instance = count_instance(cat_raw_filtered, num_raw_filtered)
  for cur_th in th_range:
    REPARTITION_CONST = 200
    cat_filtered = availability_filter(cat_raw_filtered, num_instance, availability_th=cur_th,
                                        REPARTITION_CONST=REPARTITION_CONST)

    num_filtered = availability_filter(num_raw_filtered, num_instance, availability_th=cur_th,
                                        REPARTITION_CONST=REPARTITION_CONST)


    cat_featurized = cat_featurizer(cat_filtered, voca_df=cat_voca_list\
                                                          , REPARTITION_CONST=REPARTITION_CONST)


    num_featurized = num_featurizer(num_filtered, ref_df=num_ref_list\
                                                          , REPARTITION_CONST=REPARTITION_CONST)

    merged_all = feature_aggregator(num_featurized, cat_featurized\
                                                          , REPARTITION_CONST=REPARTITION_CONST)
    target_rdd, target_schema, feature_column = flattener_df_prep(merged_all)
    cur_df = spark.createDataFrame(target_rdd, target_schema).persist()

  return cur_df
  

In [30]:
#helper methods for run_remaining
from pyspark.sql.functions import col,when,lit
def num_cat_tagger(data_frame, inputCol="VALUE", outputCol="IS_CAT",labelCol="ITEMID",REPARTITION_CONST = None,nominal_th = 2):
        #DL0411: Output should be list of features with corresponding num/cat instances
        from pyspark.sql.functions import size,collect_set
        if REPARTITION_CONST is None:
            get_nominal_var = data_frame.repartition(labelCol).groupBy(labelCol).agg(size(collect_set(inputCol)).alias("value_cnt"))\
                         .where("value_cnt<={0}".format(nominal_th)).select(labelCol).rdd.flatMap(list).collect()
        else:
            get_nominal_var = data_frame.repartition(REPARTITION_CONST).groupBy(labelCol).agg(size(collect_set(inputCol)).alias("value_cnt"))\
                         .where("value_cnt<={0}".format(nominal_th)).select(labelCol).rdd.flatMap(list).collect()
        data_frame.show()

        if REPARTITION_CONST is None:
            ret_data_frame = data_frame.withColumn(outputCol,
                when((col(inputCol).rlike('^(?!-0?(\.0+)?(E|$))-?(0|[1-9]\d*)?(\.\d+)?(?<=\d)(E-?(0|[1-9]\d*))?$'))&
                     (~col(labelCol).isin(get_nominal_var)),lit("0"))\
                .otherwise(lit("1")))
        else:
            ret_data_frame = data_frame.withColumn(outputCol,
                when((col(inputCol).rlike('^(?!-0?(\.0+)?(E|$))-?(0|[1-9]\d*)?(\.\d+)?(?<=\d)(E-?(0|[1-9]\d*))?$'))&
                     (~col(labelCol).isin(get_nominal_var)),lit("0"))\
                .otherwise(lit("1"))).repartition(REPARTITION_CONST)
        return ret_data_frame

def cat_frequency_filter(data_frame,threshold_lb = 0,threshold_ub = 1,inputCol="VALUE",labelCol="ITEMID",REPARTITION_CONST=None):
        from pyspark.sql.functions import count,monotonically_increasing_id
        label_count = data_frame.groupBy(labelCol).agg(count("*").alias("label_count"))
        if REPARTITION_CONST is not None:
            label_count = label_count.repartition(REPARTITION_CONST).persist()
        if REPARTITION_CONST is None:
            cur_freq = data_frame.groupBy(labelCol,inputCol).agg(count("*").alias("indiv_count")).join(label_count,labelCol)\
                                .withColumn("cat_freq",col("indiv_count")/col("label_count"))
        else:
            cur_freq = data_frame.groupBy(labelCol,inputCol).agg(count("*").alias("indiv_count")).repartition(REPARTITION_CONST).join(label_count,labelCol)\
                                .withColumn("cat_freq",col("indiv_count")/col("label_count")).repartition(REPARTITION_CONST)
        cur_freq = cur_freq.where((col("cat_freq")>=threshold_lb) & (col("cat_freq")<=threshold_ub)).drop("cat_freq")
        #cur_freq.orderBy(col(labelCol),-1*col("cat_freq")).show(500)

        ret_data_frame = data_frame.join(cur_freq,[inputCol,labelCol]).drop("indiv_count").drop("label_count")

        if REPARTITION_CONST is not None:
            ret_data_frame = ret_data_frame.repartition(REPARTITION_CONST)
        ret_voca = ret_data_frame.select("ITEMID","VALUE").distinct()
        ret_voca = ret_voca.rdd.map(lambda x: (x.ITEMID,x.VALUE)).zipWithUniqueId().map(lambda x: {"idx":x[1], "ITEMID":x[0][0], "VALUE":x[0][1]}).toDF()
        if REPARTITION_CONST is not None:
            ret_voca = ret_voca.repartition(REPARTITION_CONST)

        return (ret_data_frame, ret_voca)

def num_iqr_filter(data_frame, inputCol="VALUE",labelCol="ITEMID",REPARTITION_CONST = None,sc=None):
        from pyspark.sql.window import Window
        from pyspark.sql.functions import abs,percent_rank,row_number,collect_list,udf,struct,count,avg,stddev_pop
        from pyspark.sql.types import MapType,StringType,DoubleType
        if REPARTITION_CONST is None:
            value_order = Window.partitionBy(labelCol).orderBy(col(inputCol).cast("float"))
            Q1_percentile = Window.partitionBy(labelCol).orderBy(abs(0.25-col("percentile")))
            Q2_percentile = Window.partitionBy(labelCol).orderBy(abs(0.5-col("percentile")))
            Q3_percentile = Window.partitionBy(labelCol).orderBy(abs(0.75-col("percentile")))
            percent_data_frame = data_frame.select(labelCol, inputCol, percent_rank().over(value_order).alias("percentile"))
            Q1_data_frame = percent_data_frame.withColumn("Q1_rn",row_number().over(Q1_percentile)).where("Q1_rn == 1")\
                            .select(labelCol,inputCol,lit("Q1").alias("quantile"))
            Q2_data_frame = percent_data_frame.withColumn("Q2_rn",row_number().over(Q2_percentile)).where("Q2_rn == 1")\
                            .select(labelCol,inputCol,lit("Q2").alias("quantile"))
            Q3_data_frame = percent_data_frame.withColumn("Q3_rn",row_number().over(Q3_percentile)).where("Q3_rn == 1")\
                            .select(labelCol,inputCol,lit("Q3").alias("quantile"))
            merge_all = Q1_data_frame.unionAll(Q2_data_frame).unionAll(Q3_data_frame).persist()


            udf_parse_list_to_map = udf(lambda maps: dict(list(tuple(x) for x in maps)),MapType(StringType(),StringType()))

            aggregate_quantiles = merge_all.groupBy(labelCol).agg(collect_list(struct("quantile",inputCol)).alias("quant_info"))

            aggregate_quantiles = aggregate_quantiles.select(labelCol,udf_parse_list_to_map("quant_info").alias("quant_info"))

            iqr_data_frame = aggregate_quantiles.withColumn("Q1",col("quant_info").getItem("Q1").cast("float"))\
                .withColumn("Q2",col("quant_info").getItem("Q2").cast("float"))\
                .withColumn("Q3",col("quant_info").getItem("Q3").cast("float"))

        else:
            cur_label_list = data_frame.select(labelCol).distinct().rdd.flatMap(list).collect()
            cur_iqr_list = list()
            cnt = -1
            for cur_item in cur_label_list:
                cnt = cnt+1
                data_frame.where(col(labelCol) == cur_item).registerTempTable("cur_table")
                cur_iqr = sc.sql("select {0}, percentile_approx({1},0.25) as Q1, percentile_approx({2},0.5) as Q2, percentile_approx({3},0.75) as Q3 from cur_table group by {4}".format(labelCol,inputCol,inputCol,inputCol,labelCol)).first().asDict()
                cur_iqr_list.append(cur_iqr)
                sc.catalog.dropTempView("cur_table")
                #percent_data_frame = data_frame.select(labelCol, inputCol, percent_rank().over(value_order).alias("percentile")).repartition(REPARTITION_CONST).cache().checkpoint()
            iqr_data_frame = sc.createDataFrame(cur_iqr_list).repartition(REPARTITION_CONST)


        if REPARTITION_CONST is None:
            iqr_data_frame = iqr_data_frame.withColumn("IQR",col("Q3")-col("Q1"))\
                                       .withColumn("LB",col("Q1")-1.5*col("IQR"))\
                                       .withColumn("UB",col("Q3")+1.5*col("IQR"))\
                                       .select(labelCol,"LB","UB")
        else:
            iqr_data_frame = iqr_data_frame.withColumn("IQR",col("Q3")-col("Q1"))\
                                       .withColumn("LB",col("Q1")-1.5*col("IQR"))\
                                       .withColumn("UB",col("Q3")+1.5*col("IQR"))\
                                       .select(labelCol,"LB","UB").repartition(REPARTITION_CONST).persist()
        if REPARTITION_CONST is None:

            ret_data_frame = data_frame.repartition(labelCol).join(iqr_data_frame,labelCol).where((col("LB").cast("float") <= col(inputCol).cast("float")) & (col("UB").cast("float")>=col(inputCol).cast("float")))\
                                                                 .drop("LB").drop("UB").persist()
            ref_df = ret_data_frame.repartition(labelCol).groupBy(labelCol)\
                               .agg(count(inputCol).alias("ref_count"),avg(inputCol).alias("ref_avg"),stddev_pop(inputCol).alias("ref_std")).persist()

            return (ret_data_frame, ref_df)
        else:
              ret_data_frame = data_frame.join(iqr_data_frame,labelCol).where((col("LB").cast("float") <= col(inputCol).cast("float")) & (col("UB").cast("float")>=col(inputCol).cast("float")))\
                                                                  .drop("LB").drop("UB").repartition(REPARTITION_CONST)
              ref_df = ret_data_frame.groupBy(labelCol)\
                                .agg(count(inputCol).alias("ref_count"),avg(inputCol).alias("ref_avg"),stddev_pop(inputCol).alias("ref_std")).repartition(REPARTITION_CONST)

              return (ret_data_frame, ref_df)

def count_instance(raw_feature1, raw_feature2 = None):
        from pyspark.sql.functions import collect_set,size
        raw1_distinct_instance = raw_feature1\
            .select("ID","TIME_SPAN").withColumn("T1",col("TIME_SPAN").TIME_FROM.cast("timestamp"))\
            .withColumn("T2",col("TIME_SPAN").TIME_TO.cast("timestamp")).select("ID","T1","T2")
        if raw_feature2 is not None:
            raw2_distinct_instance = raw_feature2\
                .select("ID","TIME_SPAN").withColumn("T1",col("TIME_SPAN").TIME_FROM.cast("timestamp"))\
                .withColumn("T2",col("TIME_SPAN").TIME_TO.cast("timestamp")).select("ID","T1","T2")
        else:
            raw2_distinct_instance = None
        if raw2_distinct_instance is not None:
            all_inst = raw1_distinct_instance.unionAll(raw2_distinct_instance)
        else:
            all_inst = raw1_distinct_instance

        return all_inst.distinct().groupBy("ID").agg(collect_set("T1").alias("collected_set")).select("ID",size("collected_set").alias("list_size")).rdd.map(lambda x: x.list_size).reduce(lambda a,b:a+b)

def availability_filter(data_frame,n_inst = None, availability_th = 0.80,labelCol="ITEMID",idCol="ID",timeCol="TIME_SPAN",REPARTITION_CONST=None):
        from pyspark.sql.functions import count
        if not n_inst:
            total_cnt = data_frame.select(idCol,timeCol).distinct().count()

        else:
            total_cnt = n_inst
        target_label_set = data_frame.select(idCol,timeCol,labelCol).distinct()

        if REPARTITION_CONST is None:
            target_label_set = target_label_set.groupBy(labelCol).agg((count("*")/float(total_cnt)).alias("freq"))
        else:
            target_label_set = target_label_set.repartition(REPARTITION_CONST)\
                .groupBy(labelCol).agg((count("*")/float(total_cnt)).alias("freq"))

        target_label_set.orderBy(col("freq").desc()).show()

        if REPARTITION_CONST is not None:
            target_label_set = target_label_set.repartition(REPARTITION_CONST)
        target_label_set = target_label_set.where(col("freq")>=availability_th).select(labelCol).rdd.flatMap(list).collect()
        if len(target_label_set) == 0:
            return
        ret_data_frame = data_frame.where(col(labelCol).isin(target_label_set))

        return ret_data_frame

def cat_featurizer(data_frame, voca_df, inputCol="VALUE",labelCol="ITEMID",outputCol="cat_features",REPARTITION_CONST = None):
        def prep_cat_dict(avail, pos):  # internal
            ret_dict_key = list(map(lambda x: "C_" + str(x), avail))
            ret_dict = dict(zip(ret_dict_key, [0.0] * len(ret_dict_key)))
            pos_dict_key = list(map(lambda x: "C_" + str(x), pos))
            update_dict = dict(zip(pos_dict_key, [1.0] * len(pos_dict_key)))
            ret_dict.update(update_dict)
            return ret_dict

        from pyspark.sql.functions import udf,collect_set
        from pyspark.sql.types import MapType, StringType, DoubleType
        if not data_frame:
            return

        all_var = voca_df.groupBy(labelCol).agg(collect_set("idx").alias("AVAIL_LIST"))
        ret_data_frame = data_frame.join(voca_df,[inputCol,labelCol]).drop("VALUE").withColumnRenamed("idx","VALUE")
        if REPARTITION_CONST is not None:
            ret_data_frame = ret_data_frame.repartition(REPARTITION_CONST)
        ret_data_frame = value_aggregator(ret_data_frame).join(all_var,"ITEMID")

        udf_prep_cat_dict = udf(prep_cat_dict, MapType(StringType(),DoubleType()))
        ret_data_frame = ret_data_frame.withColumn("cat_features",udf_prep_cat_dict("AVAIL_LIST","VALUE_LIST"))
        return ret_data_frame

def value_aggregator(data_frame, aggregateCols = ["ID","TIME_SPAN","ITEMID"], catmarkerCol="IS_CAT", inputCol = "VALUE", outputCol="VALUE_LIST"):
        from pyspark.sql.functions import collect_set, collect_list
        cat_data_agg_frame = data_frame.where(col(catmarkerCol) == 1).groupBy(aggregateCols+[catmarkerCol])\
                                                                     .agg(collect_set(inputCol).alias(outputCol))
        num_data_agg_frame = data_frame.where(col(catmarkerCol) == 0).groupBy(aggregateCols+[catmarkerCol])\
                                                                     .agg(collect_list(inputCol).alias(outputCol))
        return cat_data_agg_frame.unionAll(num_data_agg_frame)

def num_featurizer(data_frame, ref_df=None, featurize_process = ["summary_stat","sustainment_q"], inputCol="VALUE",labelCol="ITEMID",outputCol="num_features",REPARTITION_CONST=None):
        from pyspark.sql.functions import udf,array
        from pyspark.sql.types import StringType, DoubleType, MapType
        if not data_frame:
            return
        ret_data_frame = value_aggregator(data_frame)
        if REPARTITION_CONST is not None:
            ret_data_frame = ret_data_frame
        if "summary_stat" in featurize_process:
            udf_summary_stat = udf(calc_summary_stat,MapType(StringType(),DoubleType()))
            ret_data_frame = ret_data_frame.withColumn("summary_stat",udf_summary_stat(inputCol+"_LIST",labelCol))
            if REPARTITION_CONST is not None:
                ret_data_frame = ret_data_frame

        if "sustainment_q" in featurize_process:
            udf_sustainment_quant = udf(sustainment_quantifier,MapType(StringType(),DoubleType()))
            ret_data_frame = ret_data_frame.join(ref_df,labelCol).withColumn("sustainment_q",udf_sustainment_quant("summary_stat",labelCol,"ref_avg","ref_std","ref_count")).drop("ref_avg").drop("ref_std").drop("ref_count")
            if REPARTITION_CONST is not None:
                ret_data_frame = ret_data_frame
        udf_merge_dict_all = udf(merge_dict_all,MapType(StringType(),DoubleType()))
        ret_data_frame = ret_data_frame.withColumn(outputCol,udf_merge_dict_all(array(featurize_process)))
        return ret_data_frame

def calc_summary_stat(x,labelCol):
        import numpy as np
        cur_array = np.array(x,dtype=float)
        ret_dict = dict()
        ret_dict["N_{0}_avg".format(labelCol)] = float(np.average(cur_array))
        ret_dict["N_{0}_min".format(labelCol)] = float(np.min(cur_array))
        ret_dict["N_{0}_max".format(labelCol)] = float(np.max(cur_array))
        ret_dict["N_{0}_std".format(labelCol)] = float(np.std(cur_array))
        ret_dict["N_{0}_count".format(labelCol)] = float(np.shape(cur_array)[0])
        return ret_dict

def sustainment_quantifier(x,cur_label,ref_avg, ref_std, ref_count):
        from scipy.stats import ttest_ind_from_stats
        import numpy as np
        ret_dict = dict()
        statistic, p_val = ttest_ind_from_stats(x["N_{0}_avg".format(cur_label)], x["N_{0}_std".format(cur_label)], x["N_{0}_count".format(cur_label)], ref_avg, ref_std, ref_count, equal_var=True)
        if not np.isnan(statistic):
            if statistic > 0:
                one_tailed_pval = 1.0 - p_val/2.0
            else:
                one_tailed_pval = p_val/2.0
            ret_dict["N_{0}_TT".format(cur_label)] = float(p_val)
            ret_dict["N_{0}_LT".format(cur_label)] = float(one_tailed_pval)
        return ret_dict

def merge_dict_all(x): #will not be used outside of the package
        ret_dict = dict()
        for cur_dict in x:
            ret_dict.update(cur_dict)
        return ret_dict

def feature_aggregator(num_features,cat_features,catinputCol="cat_features",numinputCol="num_features",aggregatorCol = ["ID","TIME_SPAN"], outputCol="feature_aggregated",idCol="ID",REPARTITION_CONST = None):
        from pyspark.sql.functions import col, udf, collect_list,rand
        from pyspark.sql.types import MapType, StringType, DoubleType

        if not num_features:
            ret_data_frame=cat_features.withColumnRenamed(catinputCol,"features")

        elif not cat_features:
            ret_data_frame=num_features.withColumnRenamed(numinputCol,"features")

        else:
            ret_data_frame = num_features.select(aggregatorCol+[numinputCol]).withColumnRenamed(numinputCol,"features")\
                                     .unionAll(cat_features.select(aggregatorCol+[catinputCol]).withColumnRenamed(catinputCol,"features"))

        if REPARTITION_CONST is not None:
            ret_data_frame = ret_data_frame.repartition(REPARTITION_CONST)

        udf_merge_dict_all = udf(merge_dict_all,MapType(StringType(),DoubleType()))

        # FROM HERE
        ret_data_frame = ret_data_frame.groupBy(aggregatorCol)
        if REPARTITION_CONST is not None:
            ret_data_frame = ret_data_frame.agg(collect_list("features").alias("features")).repartition(REPARTITION_CONST)

        else:
            ret_data_frame = ret_data_frame.agg(collect_list("features").alias("features"))
        #ret_data_frame.orderBy(rand()).show(truncate=False)
        ret_data_frame = ret_data_frame.withColumn(outputCol, udf_merge_dict_all("features"))
        return ret_data_frame

def flattener_df_prep(data_frame, descCol=["ID","TIME_SPAN"] ,inputCol="feature_aggregated",drop_cnt=True):
        from pyspark.sql import Row
        from pyspark.sql.functions import col,udf,lit
        from pyspark.sql.types import StructType,StructField,StringType,DoubleType,BooleanType
        data_frame.show()
        desc_schema = data_frame.select(descCol).schema
        all_feature_column = list(data_frame.select(inputCol).rdd.map(lambda x: set(x[inputCol].keys())).reduce(lambda a,b: a.union(b)))
        ret_df = data_frame.rdd.map(lambda x: x.asDict()).map(lambda cur_item: [dict((cur_col,cur_item[cur_col]) for cur_col in descCol)]\
                                                                                + [cur_item[inputCol]]).map(lambda x: merge_dict_all(x))
        inst_count = data_frame.count()
        ret_schema = desc_schema
        ret_feature_col = list()
        key_checker = udf(check_key_in_dict,BooleanType())
        for cur_col in all_feature_column:
    #        print ("{0}//{1}//{2}".format(data_frame.where(key_checker(inputCol,lit(cur_col))).count(),inst_count,cur_col))
    #        if (data_frame.where(key_checker(inputCol,lit(cur_col))).count() == inst_count):
    #            continue
            if drop_cnt:
                if cur_col.find("count") != -1:
                    continue
            ret_feature_col.append(cur_col)
            ret_schema = ret_schema.add(StructField(cur_col,DoubleType(),True))
        return (ret_df, ret_schema,ret_feature_col)

def check_key_in_dict(target_dict,target_key):
        return target_dict.has_key(target_key)

In [31]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
spark = SparkSession(SparkContext.getOrCreate())
th_range = [0.5]
cur_preprocessed = run_preprocessor(obs_df)

+------+-----+
|ITEMID|  cnt|
+------+-----+
|220045|72645|
|220210|72133|
|220277|70084|
|220181|46754|
|220179|46627|
|220180|46584|
|220052|20396|
|220050|20301|
|220051|20295|
|223761|16193|
|220074|11504|
|223835| 8892|
|225664| 7760|
|220047| 7308|
|220046| 7305|
|223770| 7243|
|223769| 7228|
|224161| 7166|
|224162| 7165|
|220339| 6741|
|223834| 6665|
|226253| 6579|
|224697| 6423|
|224687| 6413|
|220293| 6180|
|220292| 6164|
|224685| 6164|
|223876| 6158|
|223873| 6136|
|224695| 6054|
|223752| 5876|
|223751| 5875|
|223874| 5850|
|224689| 5842|
|223875| 5486|
|224690| 4623|
|227442| 4496|
| 50971| 4476|
|220545| 4406|
| 51221| 4390|
|220645| 4354|
| 50983| 4332|
|220602| 4328|
| 50902| 4304|
|220621| 4291|
|227443| 4176|
|225624| 4171|
|220615| 4163|
| 50882| 4153|
|227073| 4150|
| 51006| 4146|
| 50912| 4141|
| 50868| 4128|
| 50931| 4082|
|220635| 3993|
| 50960| 3985|
|224688| 3910|
|224738| 3905|
|224684| 3783|
|223872| 3735|
|225677| 3710|
| 50970| 3698|
|225625| 3687|
| 50893| 3

In [32]:
x = cur_preprocessed.toPandas()
print(x.head())
len(x)

         ID  ... N_220545_TT
0  130108.0  ...    0.290789
1  130108.0  ...    0.722685
2  118007.0  ...    0.840266
3  183762.0  ...    0.401749
4  143047.0  ...    0.022046

[5 rows x 350 columns]


3104

In [45]:
def annotate_dataset(action_df, cur_df, terminal_outcome, annotation_method = "p_val", sel_top = 5):
    '''
    :param cur_df:
    :param annotation_method:
    :param postfix:
    :param cur_top:
    :param non_feature_column:
    :param add_flag:
    :return:
    '''
    if annotation_method == "p_val":
        return annotate_pval_dataset(action_df, cur_df, terminal_outcome, sel_top = sel_top)

    return 
def annotate_pval_dataset(action_df, cur_df, terminal_outcome, non_feature_column = ["ID", "TIME_SPAN"], sel_top = 5):
    import pyspark
    from pyspark.ml.feature import VectorAssembler
    obs_df = cur_df
    itemid = "ITEMID"
    cur_cols = obs_df.columns
    for i in non_feature_column:
        cur_cols.remove(i)

    cur_cols = sorted(cur_cols)
    import json
    #json.dump({"non_demo_features":cur_cols},open(self.json_feature_dump_loc,"w"))
    obs_df = VectorAssembler(inputCols=cur_cols, outputCol="features_imputed").transform(obs_df)

    cur_time_list = obs_df.select("ID","TIME_SPAN")
    of_annotated = obs_df
    of_excl_training = dict()
    demo_feature = add_demo()
    #from pyspark.context import SparkContext
    #from pyspark.sql.session import SparkSession
    #sc = SparkContext.getOrCreate();
    #spark = SparkSession(sc)
    #action_df.TIME_OBS = action_df.TIME_OBS.astype(str)
    #demo_feature = spark.createDataFrame(demo_feature)
    
    of_annotated = VectorAssembler(inputCols=["features_imputed","demo_feature"],outputCol="features").transform(of_annotated.join(demo_feature,"ID"))
    from pyspark.sql.functions import col,lit,when
    cur_test_ids = get_target_test_id()
    tr_inst,te_inst = prep_TR_TE(of_annotated,test_id_list = cur_test_ids)
    train_data_ID = tr_inst.select("ID").distinct().rdd.flatMap(list).collect()

    testing_data_ID = te_inst.select("ID").distinct().rdd.flatMap(list).collect()

    train_action_df = action_df.where(col("ID").isin(train_data_ID)).persist()
    train_terminal_outcome = terminal_outcome.where(col("ID").isin(train_data_ID)).persist()
    intv_w_p_val = identify_relevant_action(train_action_df, train_terminal_outcome, tr_inst.select("ID").distinct().count(), terminal_outcome)
    #intv_w_p_val.join(def_df[def_df["SOURCE"].isin(["CPT","MED","PROC"])], itemid).orderBy("p_val").show(100, truncate=False)
    from pyspark.sql.functions import sum,rand,max,lit
    from pyspark.ml.feature import VectorAssembler
    annot_df = action_df.join(terminal_outcome,"ID").persist()
    pos_inst_dict = dict()
    from pyspark.sql.functions import count
    cur_annot_topk = sel_top
    for cur_of in [target_disch_col]:
        # For debug purpose, pass if target_of is not identified
        intv_w_p_val.where("DISCH_DX == '{0}'".format(cur_of)).orderBy(col("p_val").cast("double")).show(50,truncate=False)
        target_annot_criteria = intv_w_p_val.where("DISCH_DX == '{0}'".format(cur_of)).orderBy(col("p_val").cast("double")).limit(cur_annot_topk)
       # target_annot_criteria.write.save(self.annot_intv_dir.format(cur_of,cur_annot_topk),mode="overwrite")
        target_annot_criteria = target_annot_criteria.select(itemid).rdd.flatMap(list).collect()
        if len(target_annot_criteria) == 0:
            pos_inst_dict[cur_of] = None
            continue
        print("checkpoint 10")
        #def_df[def_df["ITEMID"].isin(target_annot_criteria)].show(cur_annot_topk,truncate=False)
        print("checkpoint 11")
        pos_inst_dict[cur_of] = annot_df.where((col(itemid).isin(target_annot_criteria)) & (col("DISCH_DX") == cur_of))\
            .select("ID", col("TIME_OBS").cast("date").alias("TIME_OBS"), lit("1").cast("double").alias("{0}_label".format(cur_of)))\
            .distinct().persist()
        print("checkpoint 12")
        #pos_inst_dict[cur_of].groupBy("{0}_label".format(cur_of)).agg(count("*")).show()
        from pyspark.sql.functions import broadcast

        true_inst = annot_df.where((col(itemid).isin(target_annot_criteria)) & (col("DISCH_DX") == cur_of))
        excl_id = annot_df.withColumn("IS_TARGET_OF",when(col("DISCH_DX") ==cur_of,lit("1").cast("double")).otherwise(lit("0").cast("double")))\
            .withColumn("IS_REL_INTV", when(col(itemid).isin(target_annot_criteria), lit("1").cast("double")).otherwise(lit("0").cast("double")))\
            .groupBy("ID").agg(sum("IS_TARGET_OF").alias("SUM_IS_TARGET_OF"),sum("IS_REL_INTV").alias("SUM_IS_REL_INTV"))\
            .where("(SUM_IS_TARGET_OF <> 0) AND (SUM_IS_REL_INTV == 0)").select("ID").distinct().rdd.flatMap(list).collect()
        print("checkpoint 13")
        tr_inst = tr_inst.withColumn("TIME_OBS",col("TIME_SPAN.TIME_TO").cast("date"))\
            .withColumn("{0}_excl".format(cur_of), col("ID").isin(excl_id).cast("double")).repartition("ID","TIME_OBS")\
            .join(broadcast(pos_inst_dict[cur_of]),["ID","TIME_OBS"],"left_outer").fillna(value=0.0,subset=["{0}_label".format(cur_of)]).persist()
        print(tr_inst.count())
        #tr_inst.groupBy("{0}_label".format(cur_of),"{0}_excl".format(cur_of)).agg(count("*")).show()
        te_inst = te_inst.withColumn("TIME_OBS",col("TIME_SPAN.TIME_TO").cast("date"))\
            .withColumn("{0}_excl".format(cur_of), col("ID").isin(excl_id).cast("double")).repartition("ID","TIME_OBS")\
            .join(broadcast(pos_inst_dict[cur_of]),["ID","TIME_OBS"],"left_outer").fillna(value=0.0, subset=["{0}_label".format(cur_of)]).persist()
        print(te_inst.count())
        #te_inst.groupBy("{0}_label".format(cur_of),"{0}_excl".format(cur_of)).agg(count("*")).show()

        tr_inst.groupBy("ID").agg(max("{0}_label".format(cur_of)).alias("{0}_label".format(cur_of)),max("{0}_excl".format(cur_of)).alias("{0}_excl".format(cur_of))).groupBy("{0}_label".format(cur_of),"{0}_excl".format(cur_of)).agg(count("*")).show()
        te_inst.groupBy("ID").agg(max("{0}_label".format(cur_of)).alias("{0}_label".format(cur_of)),max("{0}_excl".format(cur_of)).alias("{0}_excl".format(cur_of))).groupBy("{0}_label".format(cur_of),"{0}_excl".format(cur_of)).agg(count("*")).show()

    #tr_inst.write.save(self.training_temp_dir, mode="overwrite")
    #te_inst.write.save(self.testing_temp_dir, mode="overwrite")

    #tr_inst = self.spark.read.parquet(self.training_temp_dir)
    #te_inst = self.spark.read.parquet(self.testing_temp_dir)
    #te_inst.show()
    return (tr_inst,te_inst)

def identify_relevant_action(action_df, terminal_df,cnt_pop, terminal_outcome):
        from pyspark.sql.functions import col
        def ret_p_val(obs,cnt_pop,p):
            from scipy.stats import binom_test
            return float(binom_test(obs,cnt_pop,p,alternative='greater'))

        from pyspark.sql.functions import count, udf,col
        from pyspark.sql.types import DoubleType

        cur_dx_cnt = terminal_outcome.groupBy("DISCH_DX").agg(count("*").alias("DISCH_DX_CNT")).cache()
        distinct_action_df = action_df.select("ID","ITEMID").distinct().cache()
        agg_action_df = distinct_action_df.groupBy("ITEMID").agg(count("*").alias("action_cnt")).withColumn("action_prop",col("action_cnt")/float(cnt_pop))
        merged_terminal_action = terminal_df.join(distinct_action_df,"ID").groupBy("DISCH_DX","ITEMID").agg(count("*").alias("DISCH_DX_ACTION_CNT")).join(cur_dx_cnt,"DISCH_DX").join(agg_action_df,"ITEMID").persist()

        udf_binom_test = udf(ret_p_val,DoubleType())
        cur_test = merged_terminal_action.withColumn("p_val",udf_binom_test("DISCH_DX_ACTION_CNT","DISCH_DX_CNT","action_prop")).cache()
        return cur_test
def prep_TR_TE(merged_df, per_instance=False, tr_prop=0.9,targetCol="ID",test_id_list = []):
    from pyspark.sql.functions import col
    if len(test_id_list) != 0:
        tr_inst = merged_df.where(~col(targetCol).isin(test_id_list))
        te_inst = merged_df.where(col(targetCol).isin(test_id_list))
        return (tr_inst,te_inst)
    if per_instance:
        tr_inst, te_inst = merged_df.randomSplit([tr_prop, 1-tr_prop])
    else:
        tr_id, te_id = merged_df.select(targetCol).distinct().randomSplit([tr_prop,1-tr_prop])
        tr_id = tr_id.rdd.flatMap(list).collect()
        te_id = te_id.rdd.flatMap(list).collect()
        tr_inst = merged_df.where(col(targetCol).isin(tr_id))
        te_inst = merged_df.where(col(targetCol).isin(te_id))
    return (tr_inst,te_inst)

def run_RF(tr_inst,te_inst, target_disch_col, eval_performance_criteria = "AUROC", eval_cv_or_tvt = "CV", model_of = [], cur_cv_fold = 2):
        from pyspark.sql.functions import col

        if model_of == []:
            model_of = target_disch_col
        if type(model_of) == str:
            model_of = [model_of]

        from pyspark.ml.classification import GBTClassifier as cur_model_selection

        cur_classifier = cur_model_selection(featuresCol="features",checkpointInterval=5)

        from pyspark.ml import Pipeline
        from pyspark.ml.evaluation import BinaryClassificationEvaluator
        from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

        if eval_performance_criteria == "AUPRC":
            target_metric="areaUnderPR"
        elif eval_performance_criteria == "AUROC":
            target_metric="areaUnderROC"
        else:
            raise Exception("eval_metric should be either 'AUPRC' or 'AUROC'")


        evaluator = BinaryClassificationEvaluator(metricName=target_metric)
        paramGrid = get_param_grid(cur_model_selection) ########

        if eval_cv_or_tvt == "CV":
            pipeline = Pipeline(stages=[cur_classifier])
            orig_tr_inst = tr_inst
            orig_te_inst = te_inst
            from pyspark.sql.functions import count, datediff
            from pyspark.sql.functions import udf, log, sum, exp, max
            udf_prob = udf(lambda x: x.toArray().tolist()[1])
            from pyspark.sql.functions import corr, udf, isnan
            for cur_of in model_of:
                #should move this to back
                te_inst = orig_te_inst.withColumn("label", col("{0}_label".format(cur_of)).cast("double"))
                te_inst.groupBy("label").agg(count("*")).show()

                tr_inst.printSchema()
                tr_val_pts_dict = get_target_tr_val_id() #####
                orig_tr_inst = tr_inst

                tr_pts = tr_val_pts_dict["TR"]
                val_pts = tr_val_pts_dict["VAL"]

                all_training_ids = tr_pts+val_pts
                from random import shuffle
                shuffle(all_training_ids)
                import numpy as np
                print(len(all_training_ids))
                cv_id_list_full = np.array(all_training_ids)
                perform_dict = dict()
                for cur_cv_stage in range(cur_cv_fold):
                    tr_pts = cv_id_list_full[np.linspace(0,cv_id_list_full.shape[0]-1,cv_id_list_full.shape[0])%cur_cv_fold != cur_cv_stage].tolist()
                    val_pts = cv_id_list_full[np.linspace(0,cv_id_list_full.shape[0]-1,cv_id_list_full.shape[0])%cur_cv_fold == cur_cv_stage].tolist()
                    print(np.linspace(0,cv_id_list_full.shape[0]-1,cv_id_list_full.shape[0])%cur_cv_fold == cur_cv_stage)
                    print(cv_id_list_full[np.linspace(0,cv_id_list_full.shape[0]-1,cv_id_list_full.shape[0])%cur_cv_fold == cur_cv_stage])
                    print("VAL_ROUND_{0}_TARGET IDS:{1}".format(cur_cv_stage,val_pts))

                    tr_inst = orig_tr_inst.where(col("ID").isin(tr_pts))#.persist()
                    val_inst = orig_tr_inst.where(col("ID").isin(val_pts))#.persist()
                    tr_inst = tr_inst.where(col("{0}_excl".format(cur_of)) == 0).withColumn("label", col(
                        "{0}_label".format(cur_of)).cast("double"))
                    val_inst = val_inst.withColumn("label", col(
                        "{0}_label".format(cur_of)).cast("double"))
                    
                    tr_inst.groupBy("label").agg(count("*")).show()
                    pipeline_models = pipeline.fit(tr_inst, params=paramGrid)

                    for cur_model in pipeline_models:
                        val_pred = cur_model.transform(val_inst)
                        agg_prob_val = val_pred.groupBy("ID").agg(max("label").alias("label"),
                                                                  sum(log(1.0 - udf_prob("Probability"))).alias(
                                                                      "inverse_log_sum")) \
                            .select("label", (1.0 - exp(col("inverse_log_sum"))).alias("rawPrediction"))
                        agg_prob_val.show(300, truncate=False)
                        cur_pr = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction", labelCol="label",
                                                               metricName=target_metric).evaluate(agg_prob_val)
                        if str(cur_model.stages[-1].extractParamMap()) not in perform_dict:
                            perform_dict[str(cur_model.stages[-1].extractParamMap())] = dict()
                            perform_dict[str(cur_model.stages[-1].extractParamMap())]["PERF"]=list()
                            perform_dict[str(cur_model.stages[-1].extractParamMap())]["PARAM"] = cur_model.stages[-1].extractParamMap()

                        perform_dict[str(cur_model.stages[-1].extractParamMap())]["PERF"].append(cur_pr)

                best_pf_measure = -1
                best_pf_param = None

                for key in perform_dict:
                    import numpy as np
                    test_array = np.array(perform_dict[key]["PERF"])
                    print(key, test_array.mean(), test_array.std())
                    if best_pf_measure < test_array.mean():
                        best_pf_measure = test_array.mean()
                        best_pf_param = perform_dict[key]["PARAM"]


                print("retrain model based on best hp from CV")
                print("PERF:{0}".format(best_pf_measure))
                print("HP:{0}".format(best_pf_param))
                tr_inst = orig_tr_inst.where(col("{0}_excl".format(cur_of)) == 0).withColumn("label", col(
                    "{0}_label".format(cur_of)).cast("double"))

                bestModel = pipeline.fit(tr_inst.where(col("ID").isin(cv_id_list_full.tolist())),params=[best_pf_param])[0]

                #bestModel.save(self.model_dir_template.format(cur_of, best_pf_measure))

                prediction = bestModel.transform(te_inst)
                prediction.show()
               # prediction.write.save(self.testing_result_dest_template.format(cur_of), mode="overwrite")
                tr_result = bestModel.transform(tr_inst).withColumn("Prob",udf_prob("Probability"))
                #tr_result.write.save(self.training_result_dest_template.format(cur_of), mode="overwrite")
                return prediction, tr_result

def get_param_grid(cur_model_selection):
        from pyspark.ml.tuning import ParamGridBuilder
        return ParamGridBuilder() \
            .addGrid(cur_model_selection.maxDepth, [2]) \
            .addGrid(cur_model_selection.subsamplingRate, [0.3,0.8]) \
            .addGrid(cur_model_selection.maxIter, [2]) \
            .build()
def handle_missing(cur_df, non_feature_col = ["ID", "TIME_SPAN"]):
      import pyspark
      cur_cols = cur_df.columns
      categorical_cols = list()
      numerical_cols = list()
      for i in non_feature_col:
          cur_cols.remove(i)
      for i in cur_cols:
          if i.find("C_") == 0:
              categorical_cols.append(i)
          else:
              numerical_cols.append(i)

      cur_df = cur_df.fillna(0,subset = categorical_cols).repartition(400)

      from pyspark.ml.feature import Imputer
      imputedCols = ["imp_{0}".format(x) for x in numerical_cols]
      imputer = Imputer(inputCols = numerical_cols,outputCols = imputedCols).setStrategy("mean")
      imputer_model = imputer.fit(cur_df)
      ret_data_frame = imputer_model.transform(cur_df)
      #ret_data_frame.select(non_feature_col+imputedCols+categorical_cols).show()
      #ret_data_frame.select(non_feature_col+imputedCols+categorical_cols).write.save(self.temp_missing_drop)
      #ret_data_frame = self.spark.read.parquet(self.temp_missing_drop)
      return ret_data_frame.select(non_feature_col+imputedCols+categorical_cols)
def define_and_normalize_terminal_df(action_df, def_df):
      '''
      exclude action df that doesn't fit to current criteria
      :return:
      '''
      from pyspark.context import SparkContext
      from pyspark.sql.session import SparkSession
      sc = SparkContext.getOrCreate();
      spark = SparkSession(sc)
      action_df.TIME_OBS = action_df.TIME_OBS.astype(str)
      action_df = spark.createDataFrame(action_df)

      cur_action_col = action_df
      cur_def = def_df

      original_def = cur_def
      all_itemid =get_action_itemids()
      return action_df.join(all_itemid,["ITEMID","SOURCE"]).persist()

def flatten_terminal_outcome(action_df, cur_terminal_df, target_disch_col, target_disch_icd9):
      from pyspark.sql.functions import max,col,lit
      from pyspark.context import SparkContext
      from pyspark.sql.session import SparkSession
      sc = SparkContext.getOrCreate();
      spark = SparkSession(sc)
      action_df.TIME_OBS = action_df.TIME_OBS.astype(str)
      action_df = spark.createDataFrame(action_df)
      
      terminal_action = action_df.select("ID", "ITEMID").distinct()
      cur_terminal_df = spark.createDataFrame(cur_terminal_df)
      terminal_outcome = cur_terminal_df
      from pyspark.sql.functions import when
      cur_of_col = [target_disch_col]
      terminal_outcome = terminal_outcome.withColumn(target_disch_col,when(col("ICD9_CODE").isin(target_disch_icd9),lit("1")).otherwise(lit("0")))\
          .select(["ID"] + cur_of_col).groupBy("ID").agg(
          *(max(c).alias(c) for c in cur_of_col)).select(
          *(col(c).cast("double").alias(c) if c in cur_of_col else col(c) for c in ["ID"] + cur_of_col))
      raw_terminal_outcome = terminal_outcome
      from pyspark.sql.functions import rand
      #cms_dx_def = self.cur_annotator.annotate_of_label()

      target_terminal_outcome_table = terminal_outcome
      flatten_of_list = list()
      for cur_of in [target_disch_col]:
          flatten_of_list.append(terminal_outcome.where("{0} == 1".format(cur_of)).select("ID",lit("{0}".format(cur_of)).alias("DISCH_DX")))



      from functools import reduce as f_reduce
      from pyspark.sql import DataFrame
      terminal_outcome = f_reduce(DataFrame.union, flatten_of_list).persist()
      return terminal_outcome


In [46]:
#work on run_experiment
def run_experiment(action_df, cur_terminal_df, target_disch_col, target_disch_icd9, cur_preprocesssed, num_intv = 5):
        '''
        print agg_prob, corr_predicted_risks and return following
        :return: dict tr_instance df, dict te_instance df, dict model(CVModel)
        '''
        from pyspark.ml import PipelineModel
        import pyspark
        #self.set_top_intv_k(num_intv) -> does nothing - >Edit NEED TO INCLUDE FOR MULTIPLE INTERVENTIONS in the annotate_dataset method
        sel_top = num_intv
        terminal_outcome = flatten_terminal_outcome(action_df, cur_terminal_df, target_disch_col, target_disch_icd9)
        from glob import glob
        tr_result = dict()
        te_result = dict()
        ret_model = dict()
        for cur_of in [target_disch_col]:
            def_df = get_def_df()
            action_df = define_and_normalize_terminal_df(action_df, def_df)
            cur_df = cur_preprocessed 
            preprocessed_data = handle_missing(cur_df, non_feature_col = ["ID", "TIME_SPAN"]) 
            tr_inst, te_inst = annotate_dataset(action_df, preprocessed_data, terminal_outcome, sel_top = sel_top)
            tr_inst.show()
            te_inst.show()
            return run_RF(tr_inst, te_inst, target_disch_col, model_of = cur_of)

            #cur_model = glob(self.model_dir_template.format(cur_of,"*"))
            #print(cur_model)

            #cur_model = cur_model[0]
            #ret_model[cur_of] = PipelineModel.load(cur_model)

            #tr_result[cur_of] = self.spark.read.parquet(self.training_result_dest_template.format(cur_of))
            #te_result[cur_of] = self.spark.read.parquet(self.testing_result_dest_template.format(cur_of))

        return tr_result, te_result#, cur_model

In [47]:
#double check whether this is correct
#target disch is what is passed into data_run_experiment init
# target disch icd9 is what is passed into mimic_hp_training_scale init
#change back frorm only 1 icd9
target_disch = ["51881", "51851", "51884", "51853"]
target_disch_col = "DISCH_{0}".format("_".join(target_disch))
target_disch_icd9 = ["51881", "51851", "51884", "51853"]

In [51]:
#tr_result and te_result are backwards
#deleted cell output as it was very long
# REMEMBER TO USE 5 INTERVENTIONS WHEN CALLING RUN_EXPERIMENT -> USE A FOR LOOP PASSING numintv as the looping variable
for cur_intv_num in [10]:
  tr_result, te_result = run_experiment(action_df, terminal_df, target_disch_col, target_disch_icd9, cur_preprocessed, num_intv = cur_intv_num)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (4,5,7,11) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


37878


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:42: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 330712483 entries, 0 to 330712482
Data columns (total 5 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   ID        int32 
 1   ITEMID    int32 
 2   TIME_OBS  object
 3   VALUE     object
 4   SOURCE    object
dtypes: int32(2), object(3)
memory usage: 9.9+ GB
None
Index              128
ID          1322849932
ITEMID      1322849932
TIME_OBS    2645699864
VALUE       2645699864
SOURCE      2645699864
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27854055 entries, 0 to 27854054
Data columns (total 5 columns):
 #   Column    Dtype  
---  ------    -----  
 0   ID        float64
 1   ITEMID    int64  
 2   TIME_OBS  object 
 3   VALUE     object 
 4   SOURCE    object 
dtypes: float64(1), int64(1), object(3)
memory usage: 1.0+ GB
None
Index             128
ID          222832440
ITEMID      222832440
TIME_OBS    222832440
VALUE       222832440
SOURCE      222832440
dtype: int64
check!!!
(0, 1)
         ID  ITEMID  

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:170: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 330712483 entries, 0 to 330712482
Data columns (total 5 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   ID        int32 
 1   ITEMID    int32 
 2   TIME_OBS  object
 3   VALUE     object
 4   SOURCE    object
dtypes: int32(2), object(3)
memory usage: 9.9+ GB
None
Index              128
ID          1322849932
ITEMID      1322849932
TIME_OBS    2645699864
VALUE       2645699864
SOURCE      2645699864
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27854055 entries, 0 to 27854054
Data columns (total 5 columns):
 #   Column    Dtype  
---  ------    -----  
 0   ID        float64
 1   ITEMID    int64  
 2   TIME_OBS  object 
 3   VALUE     object 
 4   SOURCE    object 
dtypes: float64(1), int64(1), object(3)
memory usage: 1.0+ GB
None
Index             128
ID          222832440
ITEMID      222832440
TIME_OBS    222832440
VALUE       222832440
SOURCE      222832440
dtype: int64
check!!!
(0, 1)
         ID  ITEMID  

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  del sys.path[0]


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 330712483 entries, 0 to 330712482
Data columns (total 5 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   ID        int32 
 1   ITEMID    int32 
 2   TIME_OBS  object
 3   VALUE     object
 4   SOURCE    object
dtypes: int32(2), object(3)
memory usage: 9.9+ GB
None
Index              128
ID          1322849932
ITEMID      1322849932
TIME_OBS    2645699864
VALUE       2645699864
SOURCE      2645699864
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27854055 entries, 0 to 27854054
Data columns (total 5 columns):
 #   Column    Dtype  
---  ------    -----  
 0   ID        float64
 1   ITEMID    int64  
 2   TIME_OBS  object 
 3   VALUE     object 
 4   SOURCE    object 
dtypes: float64(1), int64(1), object(3)
memory usage: 1.0+ GB
None
Index             128
ID          222832440
ITEMID      222832440
TIME_OBS    222832440
VALUE       222832440
SOURCE      222832440
dtype: int64
check!!!
(0, 1)
         ID  ITEMID  

In [50]:
for i in [5]:
  print(i)

5


In [52]:
#training and testing raw accuracy from Dae's code (GBT w/ 5-fold CV)
#tr_resultdf became te_result to corrrect for backwards return, can verify by checking the length
tr_resultdf = te_result.toPandas()
cor = 0
for i in range(len(tr_resultdf)):
  if(tr_resultdf["prediction"][i] == tr_resultdf["label"][i]):
    cor += 1
print("training accuracy :" + str(cor / len(tr_resultdf)))
te_resultdf = tr_result.toPandas()
cor = 0
for i in range(len(te_resultdf)):
  if(te_resultdf["prediction"][i] == te_resultdf["label"][i]):
    cor += 1
print("testing accuracy: " + str(cor / len(te_resultdf)))
print("len training: " + str(len(tr_resultdf)))
print("len testing: " + str(len(te_resultdf)))

training accuracy :0.9351214208046393
testing accuracy: 0.735202492211838
len training: 2759
len testing: 321


In [53]:
from pyspark.ml.linalg import DenseVector
#double check whether to take from label column name or the other one -> perhaps try both and see which is better (that should be the combined one), the .map is to uncondense Dense Vectorss
X_train = tr_resultdf["features_imputed"]
X_test = te_resultdf["features_imputed"]
y_train = tr_resultdf["label"]
y_test = te_resultdf["label"]
X_train = X_train.map(lambda vector: vector.toArray())
X_test = X_test.map(lambda vector: vector.toArray())

In [54]:
#vstack to increase shape for sklearn / tf
#double check if training set split is supposed to be this large (closer to 90 rather than 70-80)
import numpy as np
X_train = np.vstack(X_train)
X_test = np.vstack(X_test)
print(X_train.shape)
X_test.shape

(2759, 348)


(321, 348)

In [55]:
def print_counts(y_actual, y_hat):
    TP = 0
    FP = 0
    TN = 0
    FN = 0
    for i in range(len(y_hat)): 
        if y_actual[i]==y_hat[i]==1:
           TP += 1
        if y_hat[i]==1 and y_actual[i]!=y_hat[i]:
           FP += 1
        if y_actual[i]==y_hat[i]==0:
           TN += 1
        if y_hat[i]==0 and y_actual[i]!=y_hat[i]:
           FN += 1

    print("true positive count: " + str(TP))
    print("false positive count: " + str(FP))
    print("true negative count: " + str(TN))
    print("false negative count: " + str(FN))
    if(TP + FP == 0):
      precision = 1
    else:
      precision = TP/ (TP + FP)
    recall = TP / (TP + FN)
    print("precision = " + str(precision))
    print("recall = " + str(recall))
    print("F1 = " + str(2*((precision * recall) / (precision + recall))))

## Attempting Machine Learning and Deep Learning Models W/O Class Weights or Oversampling

In [56]:
import sklearn
from sklearn.linear_model import LogisticRegressionCV
model = LogisticRegressionCV(cv = 10)
model.fit(X_train, y_train)
preds = model.predict(X_test)
from sklearn.metrics import f1_score, recall_score, precision_score 
count = 0
for i in range(len(preds)):
  if(preds[i] == y_test[i]):
    count += 1
print("raw acc " + str(count / len(y_test)))
print_counts(y_test, preds)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


raw acc 0.7632398753894081
true positive count: 23
false positive count: 28
true negative count: 222
false negative count: 48
precision = 0.45098039215686275
recall = 0.323943661971831
F1 = 0.3770491803278689


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [57]:
import sklearn
from sklearn.svm import SVC
model = SVC()
model.fit(X_train, y_train)
preds = model.predict(X_test)
from sklearn.metrics import f1_score, precision_score, recall_score
count = 0
for i in range(len(preds)):
  if(preds[i] == y_test[i]):
    count += 1
print("raw acc " + str(count / len(y_test)))
print_counts(y_test, preds)

raw acc 0.778816199376947
true positive count: 0
false positive count: 0
true negative count: 250
false negative count: 71
precision = 1
recall = 0.0
F1 = 0.0


In [58]:
import sklearn
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(n_neighbors= 5)
model.fit(X_train, y_train)
preds = model.predict(X_test)
count = 0
for i in range(len(preds)):
  if(preds[i] == y_test[i]):
    count += 1
print("raw acc " + str(count / len(y_test)))
print_counts(y_test, preds)

raw acc 0.7165109034267912
true positive count: 14
false positive count: 34
true negative count: 216
false negative count: 57
precision = 0.2916666666666667
recall = 0.19718309859154928
F1 = 0.2352941176470588


In [59]:
import sklearn
from sklearn.gaussian_process import GaussianProcessClassifier
model = GaussianProcessClassifier()
model.fit(X_train, y_train)
preds = model.predict(X_test)
count = 0
for i in range(len(preds)):
  if(preds[i] == y_test[i]):
    count += 1
print("raw acc " + str(count / len(y_test)))
print_counts(y_test, preds)

raw acc 0.7632398753894081
true positive count: 0
false positive count: 5
true negative count: 245
false negative count: 71
precision = 0.0
recall = 0.0


ZeroDivisionError: ignored

In [60]:
import sklearn
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()
model.fit(X_train, y_train)
preds = model.predict(X_test)
count = 0
for i in range(len(preds)):
  if(preds[i] == y_test[i]):
    count += 1
print("raw acc " + str(count / len(y_test)))
print_counts(y_test, preds)

raw acc 0.6479750778816199
true positive count: 20
false positive count: 62
true negative count: 188
false negative count: 51
precision = 0.24390243902439024
recall = 0.28169014084507044
F1 = 0.261437908496732


In [61]:
import sklearn
from sklearn.ensemble import GradientBoostingClassifier
model = GradientBoostingClassifier()
model.fit(X_train, y_train)
preds = model.predict(X_test)
from sklearn.metrics import f1_score
count = 0
for i in range(len(preds)):
  if(preds[i] == y_test[i]):
    count += 1
print("raw acc " + str(count / len(y_test)))
print_counts(y_test, preds)

raw acc 0.7320872274143302
true positive count: 15
false positive count: 30
true negative count: 220
false negative count: 56
precision = 0.3333333333333333
recall = 0.2112676056338028
F1 = 0.2586206896551724


In [63]:
import sklearn
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(X_train, y_train)
preds = model.predict(X_test)
from sklearn.metrics import f1_score
print("precision score " + str(precision_score(preds, y_test, average='macro')))
print("recall score " + str(recall_score(preds, y_test, average='macro')))
print("f1 score " + str(f1_score(preds, y_test, average='macro')))
count = 0
for i in range(len(preds)):
  if(preds[i] == y_test[i]):
    count += 1
print("raw acc " + str(count / len(y_test)))
print_counts(y_test, preds)

precision score 0.48904225352112674
recall score 0.437459807073955
f1 score 0.4419356968376576
raw acc 0.7538940809968847
true positive count: 1
false positive count: 9
true negative count: 241
false negative count: 70
precision = 0.1
recall = 0.014084507042253521
F1 = 0.02469135802469136


In [64]:
y_trainencoded = [[] for i in range(len(y_train))] 
y_testencoded = [[] for i in range(len(y_test))] 
count = 0
for element in y_train:
  if(element == 1):
    y_trainencoded[count].append(1)
    y_trainencoded[count].append(0)
  else:
    y_trainencoded[count].append(0)
    y_trainencoded[count].append(1)
  count += 1

count2 = 0
for element in y_test:
  if(element == 1):
    y_testencoded[count2].append(1)
    y_testencoded[count2].append(0)
  else:
    y_testencoded[count2].append(0)
    y_testencoded[count2].append(1)
  count2 += 1
y_trainencoded = np.array(y_trainencoded)
y_testencoded = np.array(y_testencoded)

In [65]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models
from tensorflow.keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization, LSTM
model = Sequential()
model.add(Flatten())
model.add(Dense(500, activation = 'relu')) 
model.add(Dropout(.2))
model.add(Dense(50, activation = 'relu')) 
model.add(Dropout(.2))
model.add(Dense(2, activation = 'softmax'))

In [66]:
from keras.optimizers import Adam
model.compile(loss='binary_crossentropy',
              optimizer="Adam",
              metrics=['accuracy'])
model.fit(X_train, y_trainencoded, batch_size = 64, epochs = 10, validation_split= .2, shuffle = True)

ImportError: ignored

In [ ]:
modelOutput = model.predict(X_test)
preds = []
for element in modelOutput:
  preds.append(np.argmax(element))
from sklearn.metrics import f1_score
print("precision score " + str(precision_score(preds, y_test, average='macro')))
print("recall score " + str(recall_score(preds, y_test, average='macro')))
print("f1 score " + str(f1_score(preds, y_test, average='macro')))
count = 0
for i in range(len(preds)):
  if(preds[i] == np.argmax(y_testencoded[i])):
    count += 1
print("raw acc " + str(count / len(y_testencoded)))
tempPreds = []
for i in preds:
  tempPreds.append(np.argmax(i))
print_counts(y_test, tempPreds)

## Attempting Machine Learning and Deep Learning Models w/ Class Weights

In [73]:
from sklearn.utils import class_weight
d_class_weights = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)
d_class_weights = {i : d_class_weights[i] for i in range(2)}
d_class_weights

{0: 0.6404363974001857, 1: 2.2801652892561983}

In [74]:
import sklearn
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(class_weight= d_class_weights)
model.fit(X_train, y_train)
preds = model.predict(X_test)
count = 0
for i in range(len(preds)):
  if(preds[i] == y_test[i]):
    count += 1
print("raw acc " + str(count / len(y_test)))
print_counts(y_test, preds)

raw acc 0.5171339563862928
true positive count: 42
false positive count: 126
true negative count: 124
false negative count: 29
precision = 0.25
recall = 0.5915492957746479
F1 = 0.3514644351464435


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [75]:
import sklearn
model = SVC(class_weight= d_class_weights)
model.fit(X_train, y_train)
preds = model.predict(X_test)
count = 0
for i in range(len(preds)):
  if(preds[i] == y_test[i]):
    count += 1
print("raw acc " + str(count / len(y_test)))
print_counts(y_test, preds)

raw acc 0.5856697819314641
true positive count: 38
false positive count: 100
true negative count: 150
false negative count: 33
precision = 0.2753623188405797
recall = 0.5352112676056338
F1 = 0.3636363636363636


In [76]:
import sklearn
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier(class_weight= d_class_weights)
model.fit(X_train, y_train)
preds = model.predict(X_test)
count = 0
for i in range(len(preds)):
  if(preds[i] == y_test[i]):
    count += 1
print("raw acc " + str(count / len(y_test)))
print_counts(y_test, preds)

raw acc 0.6853582554517134
true positive count: 10
false positive count: 40
true negative count: 210
false negative count: 61
precision = 0.2
recall = 0.14084507042253522
F1 = 0.16528925619834714


In [77]:
import sklearn
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(class_weight= d_class_weights)
model.fit(X_train, y_train)
preds = model.predict(X_test)
count = 0
for i in range(len(preds)):
  if(preds[i] == y_test[i]):
    count += 1
print("raw acc " + str(count / len(y_test)))
print_counts(y_test, preds)

raw acc 0.7507788161993769
true positive count: 1
false positive count: 10
true negative count: 240
false negative count: 70
precision = 0.09090909090909091
recall = 0.014084507042253521
F1 = 0.02439024390243903


In [78]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models
from tensorflow.keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization, LSTM
model = Sequential()
model.add(Flatten())
model.add(Dense(5000, activation = 'relu')) 
model.add(BatchNormalization())
model.add(Dense(50, activation = 'relu')) 
model.add(Dense(2, activation = 'softmax'))

In [96]:
import keras.optimizers
model.compile(loss='binary_crossentropy',
              optimizer="Adam",
              metrics=['accuracy'])
model.fit(X_train, y_trainencoded, batch_size = 64, epochs = 20, validation_split= .2, shuffle = True, class_weight = d_class_weights)

Epoch 1/20
35/35 [==============================] - 1s 9ms/step - loss: 1.2821 - accuracy: 0.7775 - val_loss: 0.6718 - val_accuracy: 0.7935
Epoch 2/20
35/35 [==============================] - 0s 4ms/step - loss: 1.2550 - accuracy: 0.7775 - val_loss: 0.6625 - val_accuracy: 0.7935
Epoch 3/20
35/35 [==============================] - 0s 4ms/step - loss: 1.2288 - accuracy: 0.7775 - val_loss: 0.6536 - val_accuracy: 0.7935
Epoch 4/20
35/35 [==============================] - 0s 4ms/step - loss: 1.2033 - accuracy: 0.7775 - val_loss: 0.6451 - val_accuracy: 0.7935
Epoch 5/20
35/35 [==============================] - 0s 4ms/step - loss: 1.1789 - accuracy: 0.7775 - val_loss: 0.6368 - val_accuracy: 0.7935
Epoch 6/20
35/35 [==============================] - 0s 4ms/step - loss: 1.1551 - accuracy: 0.7775 - val_loss: 0.6290 - val_accuracy: 0.7935
Epoch 7/20
35/35 [==============================] - 0s 4ms/step - loss: 1.1322 - accuracy: 0.7775 - val_loss: 0.6216 - val_accuracy: 0.7935
Epoch 8/20
35/35 [==

In [97]:
modelOutput = model.predict(X_test)
preds = []
for element in modelOutput:
  preds.append(np.argmax(element))
from sklearn.metrics import f1_score
print("precision score " + str(precision_score(preds, y_test, average='macro')))
print("recall score " + str(recall_score(preds, y_test, average='macro')))
print("f1 score " + str(f1_score(preds, y_test, average='macro')))
count = 0
for i in range(len(preds)):
  if(preds[i] == np.argmax(y_testencoded[i])):
    count += 1
print("raw acc " + str(count / len(y_testencoded)))
tempPreds = []
for i in preds:
  tempPreds.append(np.argmax(i))
print_counts(y_test, tempPreds)

precision score 0.5
recall score 0.11059190031152648
f1 score 0.18112244897959184
raw acc 0.778816199376947
true positive count: 0
false positive count: 0
true negative count: 250
false negative count: 71
precision = 1
recall = 0.0
F1 = 0.0


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Attempting Machine Learning and Deep Learning Models w/ Simple Random Oversampling

In [80]:
X_train = tr_resultdf["features_imputed"]
X_test = te_resultdf["features_imputed"]
y_train = tr_resultdf["label"]
y_test = te_resultdf["label"]
X_train = X_train.map(lambda vector: vector.toArray())
X_test = X_test.map(lambda vector: vector.toArray())
#vstack to increase shape for sklearn / tf
X_train = np.vstack(X_train)
X_test = np.vstack(X_test)
print(X_train.shape)
X_test.shape

(2759, 348)


(321, 348)

In [81]:
import imblearn
from imblearn.over_sampling import RandomOverSampler
from collections import Counter
oversample = RandomOverSampler(sampling_strategy='minority')
X_trainoversampled, y_trainoversampled = oversample.fit_resample(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [82]:
import sklearn
model = LogisticRegression()
model.fit(X_trainoversampled, y_trainoversampled)
preds = model.predict(X_test)
count = 0
for i in range(len(preds)):
  if(preds[i] == y_test[i]):
    count += 1
print("raw acc " + str(count / len(y_test)))
print_counts(y_test, preds)

raw acc 0.5700934579439252
true positive count: 40
false positive count: 107
true negative count: 143
false negative count: 31
precision = 0.272108843537415
recall = 0.5633802816901409
F1 = 0.3669724770642202


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [83]:
import sklearn
from sklearn.svm import SVC
model = SVC()
model.fit(X_trainoversampled, y_trainoversampled)
preds = model.predict(X_test)
count = 0
for i in range(len(preds)):
  if(preds[i] == y_test[i]):
    count += 1
print("raw acc " + str(count / len(y_test)))
print_counts(y_test, preds)

raw acc 0.5887850467289719
true positive count: 40
false positive count: 101
true negative count: 149
false negative count: 31
precision = 0.28368794326241137
recall = 0.5633802816901409
F1 = 0.37735849056603776


In [84]:
import sklearn
model = KNeighborsClassifier()
model.fit(X_trainoversampled, y_trainoversampled)
preds = model.predict(X_test)
count = 0
for i in range(len(preds)):
  if(preds[i] == y_test[i]):
    count += 1
print("raw acc " + str(count / len(y_test)))
print_counts(y_test, preds)

raw acc 0.5545171339563862
true positive count: 37
false positive count: 109
true negative count: 141
false negative count: 34
precision = 0.2534246575342466
recall = 0.5211267605633803
F1 = 0.34101382488479265


In [85]:
import sklearn
model = GaussianProcessClassifier()
model.fit(X_trainoversampled, y_trainoversampled)
preds = model.predict(X_test)
count = 0
for i in range(len(preds)):
  if(preds[i] == y_test[i]):
    count += 1
print("raw acc " + str(count / len(y_test)))
print_counts(y_test, preds)

raw acc 0.7632398753894081
true positive count: 0
false positive count: 5
true negative count: 245
false negative count: 71
precision = 0.0
recall = 0.0


ZeroDivisionError: ignored

In [86]:
import sklearn
model = DecisionTreeClassifier()
model.fit(X_trainoversampled, y_trainoversampled)
preds = model.predict(X_test)
count = 0
for i in range(len(preds)):
  if(preds[i] == y_test[i]):
    count += 1
print("raw acc " + str(count / len(y_test)))
print_counts(y_test, preds)

raw acc 0.6822429906542056
true positive count: 24
false positive count: 55
true negative count: 195
false negative count: 47
precision = 0.3037974683544304
recall = 0.3380281690140845
F1 = 0.31999999999999995


In [87]:
import sklearn
model = GradientBoostingClassifier()
model.fit(X_trainoversampled, y_trainoversampled)
preds = model.predict(X_test)
count = 0
for i in range(len(preds)):
  if(preds[i] == y_test[i]):
    count += 1
print("raw acc " + str(count / len(y_test)))
print_counts(y_test, preds)

raw acc 0.6230529595015576
true positive count: 27
false positive count: 77
true negative count: 173
false negative count: 44
precision = 0.25961538461538464
recall = 0.38028169014084506
F1 = 0.3085714285714286


In [88]:
import sklearn
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(X_trainoversampled, y_trainoversampled)
preds = model.predict(X_test)
count = 0
for i in range(len(preds)):
  if(preds[i] == y_test[i]):
    count += 1
print("raw acc " + str(count / len(y_test)))
print_counts(y_test, preds)

raw acc 0.7414330218068536
true positive count: 5
false positive count: 17
true negative count: 233
false negative count: 66
precision = 0.22727272727272727
recall = 0.07042253521126761
F1 = 0.10752688172043011


In [89]:
y_trainencodedoversampled = [[] for i in range(len(y_trainoversampled))] 
y_testencoded = [[] for i in range(len(y_test))] 
count = 0
for element in y_trainoversampled:
  if(element == 1):
    y_trainencodedoversampled[count].append(1)
    y_trainencodedoversampled[count].append(0)
  else:
    y_trainencodedoversampled[count].append(0)
    y_trainencodedoversampled[count].append(1)
  count += 1

count2 = 0
for element in y_test:
  if(element == 1):
    y_testencoded[count2].append(1)
    y_testencoded[count2].append(0)
  else:
    y_testencoded[count2].append(0)
    y_testencoded[count2].append(1)
  count2 += 1
y_trainencodedoversampled = np.array(y_trainencodedoversampled)
y_testencoded = np.array(y_testencoded)

In [90]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models
from tensorflow.keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization, LSTM
model = Sequential()
model.add(Flatten())
model.add(Dense(500, activation = 'relu')) 
model.add(Dense(50, activation = 'relu')) 
model.add(Dense(2, activation = 'softmax'))

In [94]:
import keras.optimizers
model.compile(loss='binary_crossentropy',
              optimizer="Adam",
              metrics=['accuracy'])
model.fit(X_trainoversampled, y_trainencodedoversampled , batch_size = 64, epochs = 20, shuffle = True)

Epoch 1/20
68/68 [==============================] - 1s 3ms/step - loss: 2.8572 - accuracy: 0.5571
Epoch 2/20
68/68 [==============================] - 0s 3ms/step - loss: 0.7188 - accuracy: 0.6144
Epoch 3/20
68/68 [==============================] - 0s 3ms/step - loss: 0.8049 - accuracy: 0.5861
Epoch 4/20
68/68 [==============================] - 0s 3ms/step - loss: 0.7089 - accuracy: 0.6202
Epoch 5/20
68/68 [==============================] - 0s 3ms/step - loss: 0.6313 - accuracy: 0.6764
Epoch 6/20
68/68 [==============================] - 0s 3ms/step - loss: 0.6624 - accuracy: 0.6551
Epoch 7/20
68/68 [==============================] - 0s 3ms/step - loss: 0.6750 - accuracy: 0.6486
Epoch 8/20
68/68 [==============================] - 0s 3ms/step - loss: 0.6140 - accuracy: 0.6968
Epoch 9/20
68/68 [==============================] - 0s 3ms/step - loss: 0.5867 - accuracy: 0.7073
Epoch 10/20
68/68 [==============================] - 0s 3ms/step - loss: 0.6344 - accuracy: 0.6869
Epoch 11/20
68/68 [

In [95]:
modelOutput = model.predict(X_test)
preds = []
for element in modelOutput:
  preds.append(np.argmax(element))
from sklearn.metrics import f1_score
print("precision score " + str(precision_score(preds, y_test, average='macro')))
print("recall score " + str(recall_score(preds, y_test, average='macro')))
print("f1 score " + str(f1_score(preds, y_test, average='macro')))
count = 0
for i in range(len(preds)):
  if(preds[i] == np.argmax(y_testencoded[i])):
    count += 1
print("raw acc " + str(count / len(y_testencoded)))
tempPreds = []
for i in preds:
  tempPreds.append(np.argmax(i))
print_counts(y_test, tempPreds)

precision score 0.5
recall score 0.11059190031152648
f1 score 0.18112244897959184
raw acc 0.778816199376947
true positive count: 0
false positive count: 0
true negative count: 250
false negative count: 71
precision = 1
recall = 0.0
F1 = 0.0


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Preprocessing Clinical Note Features

In [98]:
import pandas as pd
df = pd.read_csv("/content/physionet.org/files/mimiciii/1.4/NOTEEVENTS.csv.gz", low_memory= False)
df.head()

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
0,174,22532,167853.0,2151-08-04,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2151-7-16**] Dischar...
1,175,13702,107527.0,2118-06-14,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2118-6-2**] Discharg...
2,176,13702,167118.0,2119-05-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2119-5-4**] D...
3,177,13702,196489.0,2124-08-18,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2124-7-21**] ...
4,178,26880,135453.0,2162-03-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2162-3-3**] D...


In [99]:
tr_resultdf.head()

,ID,TIME_OBS,TIME_SPAN,imp_N_227465_std,imp_N_225625_avg,imp_N_220179_max,imp_N_227073_avg,imp_N_51237_std,imp_N_220047_TT,imp_N_51222_LT,imp_N_220621_min,imp_N_220181_avg,imp_N_50960_LT,imp_N_51006_avg,imp_N_220180_min,imp_N_223751_LT,imp_N_227442_avg,imp_N_220046_std,imp_N_220181_TT,imp_N_220180_max,imp_N_50960_min,imp_N_51222_avg,imp_N_220046_LT,imp_N_220228_std,imp_N_227442_std,imp_N_50902_TT,imp_N_51265_min,imp_N_223761_std,imp_N_50902_LT,imp_N_50902_std,imp_N_50912_avg,imp_N_51274_max,imp_N_51248_min,imp_N_227467_max,imp_N_227073_TT,imp_N_227465_LT,imp_N_51237_avg,imp_N_223769_std,imp_N_220179_TT,imp_N_223761_min,...,imp_N_223751_avg,imp_N_220645_avg,imp_N_224162_avg,imp_N_50868_LT,imp_N_50902_max,imp_N_50912_std,imp_N_227465_avg,imp_N_220645_max,imp_N_51221_std,imp_N_227467_avg,imp_N_220210_avg,imp_N_51249_avg,imp_N_51250_LT,imp_N_220228_TT,imp_N_51237_min,imp_N_225677_std,imp_N_50983_avg,imp_N_51250_std,imp_N_223751_TT,imp_N_226253_max,imp_N_51265_TT,imp_N_51275_std,imp_N_50970_std,imp_N_51237_max,imp_N_50960_max,imp_N_223770_min,imp_N_220615_std,imp_N_51277_avg,imp_N_220047_std,imp_N_220545_TT,features_imputed,demo_feature,features,DISCH_51881_51851_51884_51853_excl,DISCH_51881_51851_51884_51853_label,label,rawPrediction,probability,prediction,Prob
0,194216.0,2125-11-06,"(2125-11-05 00:00:00, 2125-11-06 00:00:00)",0.130655,8.373489,122.000000,21.0,0.014815,0.041623,0.822296,99.000000,59.181818,0.487134,44.0,31.00000,0.426931,4.750,0.0,3.344461e-06,61.000000,2.019275,11.0,0.383198,0.0,0.35000,0.892992,247.0,0.698570,0.446496,0.000000,2.40,16.326121,29.8,1.477589,0.047244,0.460002,1.460824,0.0,2.163641e-02,96.9,...,160.416362,136.0,8.0,0.977662,103.0,0.00,16.227217,136.0,0.0,1.461156,27.708333,33.8,0.330686,0.362813,1.445393,0.089576,136.0,0.0,0.726987,85.0,0.622558,0.876946,0.091198,1.477641,2.093509,90.0,0.00,15.6,0.0,0.349556,"[0.5050020692768213, 0.9899958614463573, 85.04...","(79.64109589041095, 0.0, 1.0, 0.0, 0.0, 0.0, 0...","(0.5050020692768213, 0.9899958614463573, 85.04...",0.0,0.0,0.0,"[1.3963874412694455, -1.3963874412694455]","[0.9422841413262901, 0.057715858673709874]",0.0,0.057715858673709874
1,170247.0,2156-09-17,"(2156-09-16 00:00:00, 2156-09-17 00:00:00)",0.000000,7.600000,183.000000,9.0,0.000000,0.285276,0.540661,106.000000,101.967742,0.011833,67.0,62.00000,0.426931,4.500,0.0,7.158139e-25,100.000000,1.400000,9.8,1.000000,0.0,0.00000,0.016272,237.0,0.794090,0.991864,0.000000,1.70,13.500000,34.3,1.200000,0.185969,0.244142,1.200000,0.0,1.609251e-22,97.4,...,160.416362,143.0,8.0,0.093438,119.0,0.00,13.500000,143.0,0.0,1.200000,18.161290,35.2,0.870506,0.925646,1.200000,0.000000,143.0,0.0,0.726987,85.0,0.692057,0.000000,0.000000,1.200000,1.400000,90.0,0.00,21.4,0.0,0.878265,"[1.0, 4.086259906247097e-24, 115.4482758620689...","(47.8986301369863, 0.0, 1.0, 0.0, 0.0, 0.0, 0....","(1.0, 4.086259906247097e-24, 115.4482758620689...",0.0,0.0,0.0,"[0.5703515386897479, -0.5703515386897479]","[0.7578087014694355, 0.24219129853056454]",0.0,0.24219129853056454
2,147237.0,2152-06-13,"(2152-06-12 00:00:00, 2152-06-13 00:00:00)",0.130655,8.373489,138.940579,14.0,0.014815,0.130766,0.215207,137.000000,74.778262,0.151826,25.0,47.70464,0.426931,3.300,0.0,2.071838e-01,78.036967,1.800000,8.5,0.303425,0.0,0.20000,0.028974,73.0,0.669370,0.985513,1.500000,0.85,16.326121,29.7,1.477589,0.937650,0.460002,1.460824,0.0,1.961978e-01,97.6,...,160.416362,146.5,8.0,0.537595,115.0,0.05,16.227217,148.0,0.0,1.461156,14.781250,32.2,0.601945,0.428681,1.445393,0.500000,146.5,0.0,0.726987,85.0,0.237618,0.876946,0.500000,1.477641,1.900000,90.0,0.05,16.6,0.0,0.605820,"[0.9999999931350856, 1.3729828743819694e-08, 1...","(65.7068493150685, 1.0, 0.0, 0.0, 0.0, 0.0, 0....","(0.9999999931350856, 1.3729828743819694e-08, 1...",0.0,0.0,0.0,"[0.9471888131157044, -0.9471888131157044]","[0.8692538594617801, 0.13074614053821987]",0.0,0.13074614053821987
3,133334.0,2118-03-08,"(2118-03-07 00:00:00, 2118-03-08 00:00:00)",0.130655,7.900000,130.000000,10.0,0.014815,

In [100]:
trainingIds = tr_resultdf["ID"]
testingIds = te_resultdf["ID"]
trainingTimes = tr_resultdf["TIME_SPAN"]
testingTimes = te_resultdf["TIME_SPAN"]

In [101]:
trainingTimes[0][0]

datetime.datetime(2125, 11, 5, 0, 0)

In [102]:
df.head()

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
0,174,22532,167853.0,2151-08-04,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2151-7-16**] Dischar...
1,175,13702,107527.0,2118-06-14,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2118-6-2**] Discharg...
2,176,13702,167118.0,2119-05-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2119-5-4**] D...
3,177,13702,196489.0,2124-08-18,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2124-7-21**] ...
4,178,26880,135453.0,2162-03-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2162-3-3**] D...


In [103]:
from datetime import datetime
curId = trainingIds[2]
print(curId)
rowNoteDF = df.loc[df['HADM_ID'] == int(curId)]
rowNoteDF = rowNoteDF.reset_index()
cur = rowNoteDF["CHARTDATE"][0]
print(cur)
x = datetime(int(cur[0:4]), int(cur[5:7]), int(cur[-2:]), 1)
x
print(trainingTimes[2][1])
rowNoteDF.head(10)

147237.0
2152-06-13
2152-06-13 00:00:00


,index,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
0,3328,3181,3969,147237.0,2152-06-13,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2152-6-4**] D...
1,62775,62150,3969,147237.0,2152-06-05,NaN,NaN,Echo,Report,NaN,NaN,PATIENT/TEST INFORMATION:\nIndication: Congest...
2,109332,121541,3969,147237.0,2152-06-05,NaN,NaN,ECG,Report,NaN,NaN,Sinus rhythm\nRight bundle branch block\nLeft ...
3,109398,121591,3969,147237.0,2152-06-04,NaN,NaN,ECG,Report,NaN,NaN,Baseline artifact\nSinus tachycardia\nRight bu...
4,109399,121592,3969,147237.0,2152-06-04,NaN,NaN,ECG,Report,NaN,NaN,Sinus tachycardia. Right bundle-branch block....
5,109400,121593,3969,147237.0,2152-05-29,NaN,NaN,ECG,Report,NaN,NaN,Sinus rhythm. Left atrial abnormality. Right...
6,112469,121540,3969,147237.0,2152-06-12,NaN,NaN,ECG,Report,NaN,NaN,Sinus tachycardia\nRight bundle branch block\n...
7,324346,327813,3969,147237.0,2152-06-04,2152-06-04 09:23:00,2152-06-04 09:23:59,Physician,Physician Attending Admission Note,20066.0,NaN,Chief Complaint: DKA\n HPI:\n 65 yo woman...
8,324355,327903,3969,147237.0,2152-06-05,2152-06-05 15:14:00,2152-06-05 17:33:45,Nursing,Nursing Progress Note,14411.0,NaN,Pt is a 65 yr old adm [**2152-6-4**] from EW f...
9,324357,327907,3969,147237.0,2152-06-05,2152-06-05 09:06:00,2152-06-05 18:02:47,Physician,ICU Attending Note,20066.0,NaN,Clinician: Attending\n Hypotensive overnigh...


In [105]:
#IMPORTANT PROBLEM - UNDERSTAND WHAT NOTES TO USE USING HADM_ID -> corresponds to each patient visit, figure out what features should like (perhaps n-vector where n is each type of category note)

In [106]:
trainingText = []
for i in range(len(trainingIds)):
  curId = trainingIds[i]
  rowNoteDF = df.loc[df['ROW_ID'] == int(curId)]
  rowNoteDF = rowNoteDF.reset_index()
  cur = rowNoteDF["CHARTDATE"][0]
  noteDate = datetime(int(cur[0:4]), int(cur[5:7]), int(cur[-2:]), 1)
  print(trainingTimes[i][0])
  print(noteDate)
  print(trainingTimes[i][1])
  rowNoteDF.head()
  break
  if(trainingTimes[i][0] <= noteDate <= trainingTimes[i][1]):
    trainingText.append(rowNoteDF["TEXT"][0])
  else:
    trainingText.append([""])
    print("strange")

testingText = []
for i in range(len(testingIds)):
  curId = trainingIds[i]
  rowNoteDF = df.loc[df['ROW_ID'] == int(curId)]
  rowNoteDF = rowNoteDF.reset_index()
  testingText.append(rowNoteDF["TEXT"][0])
print(len(trainingText))
len(testingText)

2125-11-05 00:00:00
2174-07-02 01:00:00
2125-11-06 00:00:00
0


321

In [108]:
# Tokenizing inputs and saving it in tokenizedInputs
!pip install nltk
import nltk
nltk.download('punkt')
tokenizedTraining = []
for i in range(len(trainingText)):
  tokenizedTraining.append(nltk.word_tokenize(trainingText[i]))
tokenizedTesting = []
for i in range(len(testingText)):
  tokenizedTesting.append(nltk.word_tokenize(testingText[i]))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [109]:
import nltk
from nltk.tag import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer
import re, string
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
def remove_noise_and_lemmatize(tweet_tokens, stop_words = ()):
    cleaned_tokens = []
    for token, tag in pos_tag(tweet_tokens):
        token = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'\
                       '(?:%[0-9a-fA-F][0-9a-fA-F]))+','', token)
        token = re.sub("(@[A-Za-z0-9_]+)","", token)
        if tag.startswith("NN"):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'
        lemmatizer = WordNetLemmatizer()
        token = lemmatizer.lemmatize(token, pos)
        if len(token) > 0 and token not in string.punctuation and token.lower() not in stop_words:
            cleaned_tokens.append(token.lower())
    return cleaned_tokens

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [110]:
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = stopwords.words('english')
cleanedTraining = []  
for i in range(len(tokenizedTraining)):
  cleanedTraining.append(remove_noise_and_lemmatize(tokenizedTraining[i], stop_words))
cleanedTesting = []  
for i in range(len(tokenizedTesting)):
  cleanedTesting.append(remove_noise_and_lemmatize(tokenizedTesting[i], stop_words))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
